In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
from urllib.error import HTTPError
from urllib.parse import quote
from matplotlib import pyplot as plt
from tabulate import tabulate
import bs4, ssl
import pandas as pd
import re
import datetime, time # for sleep
import requests
import numpy as np
import io
import platform


import xml.etree.ElementTree as et 

import json
import urllib
import urllib.request
import sys

headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) ' \
                            'AppleWebKit/537.11 (KHTML, like Gecko) ' \
                            'Chrome/23.0.1271.64 Safari/537.11', \
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8', \
            'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3', \
            'Accept-Encoding': 'none', \
            'Accept-Language': 'en-US,en;q=0.8', \
            'Connection': 'keep-alive'}
############################################################

# pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

API_KEY='0fcb8ef56cf8e8b996d47937a8fea5aba8c8d1be'

############################################################

def __get_XML():

    xtree = et.parse("CORPCODE.xml")
    xroot = xtree.getroot() 

    df_cols = ["corp_code", "corp_name", "stock_code", "modify_date"]
    rows = []

    for node in xroot: 
        # s_name = node.attrib.get("corp_code")
        s_name = node.find("corp_code").text if node is not None else None
        s_mail = node.find("corp_name").text if node is not None else None
        s_grade = node.find("stock_code").text if node is not None else None
        s_age = node.find("modify_date").text if node is not None else None
        
        rows.append({"corp_code": s_name, "corp_name": s_mail, "stock_code": s_grade, "modify_date": s_age})

    pd.set_option('display.max_rows', 500)
    pd.set_option('display.max_columns', 500)
    pd.set_option('display.width', 1000)

    Corp_DF = pd.DataFrame(rows, columns = df_cols)
    # Corp_DF = Corp_DF.dropna(inplace=True)
    # Corp_DF.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True)
    # Corp_DF.drop(labels=['stock_code'], axis=0, index=None, columns=None, level=None, inplace=True, errors='raise')
    Corp_DF = Corp_DF.drop(Corp_DF[Corp_DF['stock_code']==' '].index, axis=0)
    # print(Corp_DF.columns)
    # print(Corp_DF.values)
    # print(Corp_DF)

    return Corp_DF

context = ssl._create_unverified_context()

def __Last_Page_Find(i, Market) :
    # print("# Last_Page_Find Start")
    if Market == 1 :
        url = "https://finance.naver.com/sise/sise_market_sum.nhn?&page=" + str(i)
    else :
        url = "https://finance.naver.com/sise/sise_market_sum.nhn?sosok=1&page=" + str(i)
    # print("# print URL")
    # print(url) # ---> 당시의 위치정보를 기반으로 15페이지 가져오는 듯, 어찌 가져오는지는 모르겠음...(2020.05.21)
    # ---> HTTP Error 403: Forbidden
    # headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}
    # req = Request(url=url, headers={'User-Agent': 'Mozilla/5.0', 'referer': 'http://m.naver.com'})

    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) ' \
                             'AppleWebKit/537.11 (KHTML, like Gecko) ' \
                             'Chrome/23.0.1271.64 Safari/537.11', \
               'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8', \
               'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3', \
               'Accept-Encoding': 'none', \
               'Accept-Language': 'en-US,en;q=0.8', \
               'Connection': 'keep-alive'}
    req = Request(url=url, headers=headers)

    try :
        html = urlopen(req).read()
        # html = urlopen(url, context=context)
    except HTTPError as err:
        if err.code == 404 :
            print("# Error HTTP : 404")
            return
        elif err.code == 403 :
            print("# Error HTTP : 403")
            return
        elif err.code == 500 :
            print("# Error HTTP : 500")
            return
        elif err.code == 505:
            print("# Error HTTP : 505")
            return
        else:
            raise

    bs_object = bs4.BeautifulSoup(html, "html.parser")
    # test = bs_object.find('table' ,attrs={'class':'type_2'})
    # numbersss = test.findAll("td", {"class": "number"})
    # print(numbersss)

    data = []
    table = bs_object.find('table', attrs={'class':'type_2'})
    table_body = table.find('tbody')
    rows = table_body.find_all('tr')
    for row in rows:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        data.append([ele for ele in cols if ele]) # Get rid of empty values
    # print("##################")
    # for x in data :
        # print(x)

    # print("##########################################")
    # print(type(data))
    Data_DF = pd.DataFrame(data, columns=['No','회사명','현재가','전일비','등락률','액면가','시가총액','상장주식수','외국인비율','거래량','PER','ROE'])
    # print(type(data_df))
    # print(data_df)
    # print("# Drop NA")
    Data_DF = Data_DF.dropna()
    Data_DF = Data_DF[['No','회사명','ROE','PER','외국인비율','현재가']]
    if Market == 1 :
        Data_DF['Market'] = "코스피"
    else :
        Data_DF['Market'] = "코스닥"
    # print("# Data_DF.LOC")
    # print(Data_DF.loc['PER':'ROE'])
    # print("# Data_DF.iLOC")
    # print(Data_DF.iloc[0:1])
    # print("# Data_DF")
    # print(Data_DF)

    return Data_DF

############################################################
# 10 페이지를 조회하여 결과 값 전달 (50기업 * 10페이지 = 500개 기업)
############################################################
def __Get_Result(Market, ROE_Min) :
    Main_DF = pd.DataFrame()
    print("# FOR Loop Start")
    print("# Market : %s" % Market)
    for i in range(1,11) : # 10 페이지 조회, 1페이지 당 50개, 총 500개???
        Data_DF = __Last_Page_Find(i, Market)
        Main_DF = Main_DF.append(Data_DF)
        # time.sleep(1)

    # 아래와 같이 종목코드를 별도로 뽑으려 했으나 Get_XML로 조치
    # df_expcode = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download', header=0)[0]
    # df_expcode = df_expcode[['회사명','종목코드']]
    # Main_DF_Result = Main_DF.join(df_expcode.set_index('회사명'), on='회사명')

    # print("DROP NA")
    # print(Main_DF)

    Main_DF_Result = Main_DF
    # Main_DF_Result = Main_DF_Result.drop(Main_DF_Result[Main_DF_Result['ROE']=='N/A'].index)
    # Main_DF_Result = Main_DF_Result.drop(Main_DF_Result[Main_DF_Result['ROE']=='NaN'].index)
    # Main_DF_Result = Main_DF_Result.drop(Main_DF_Result[Main_DF_Result['PER']=='NaN'].index)
    # ---> 잘먹지도 않고 엉뚱한 값들이 없어짐
    # Main_DF_Result.ROE = Main_DF_Result.ROE.astype(float)
    # ---> ValueError: could not convert string to float: '-1,695.71'
    Main_DF_Result.dropna(inplace=True)
    Main_DF_Result.ROE = pd.to_numeric(Main_DF_Result.ROE, errors='coerce')
    Main_DF_Result.PER = pd.to_numeric(Main_DF_Result.PER, errors='coerce')
    
    # print("# Sorting ROE")
    # Main_DF_Result = Main_DF_Result.sort_values('ROE')
    Main_DF_Result = Main_DF_Result.sort_values(by=['ROE'], ascending=False)
    print("# Main_DF_Result : Sorting ROE...")
    # print(Main_DF_Result)

    # print("######################################")
    # print(Main_DF_Result.shape())
    # print(Main_DF_Result.shape[0])
    # print(Main_DF_Result.shape[1])
    # print(Main_DF_Result.loc[(Main_DF_Result['PER'] < 20)].shape[0])
    # print(Main_DF_Result.loc[(Main_DF_Result['PER'] > 20)].shape[0])
    # print(Main_DF_Result.loc[(Main_DF_Result['PER'] > 0)& (Main_DF_Result['PER'] < 16)].shape[0])
    # print(Main_DF_Result.loc[(Main_DF_Result['PER'] > 0)& (Main_DF_Result['PER'] < 16)].head(10))
    # print(Main_DF_Result.loc[(Main_DF_Result['PER'] > 0)& (Main_DF_Result['PER'] < 16)])
    # print("Top 10")
    # Get_Comp = Main_DF_Result.loc[(Main_DF_Result['PER'] > 0)& (Main_DF_Result['PER'] < 16)].head(10)
    print("# Main_DF_Result : Sorting ROE then Condition with ROE_Min...")
    Get_Comp = Main_DF_Result.loc[(Main_DF_Result['ROE'] > ROE_Min)]
    Get_Comp = Main_DF_Result.loc[(Main_DF_Result['PER'] > 0)]
    # print(Get_Comp)

    print("# FOR Loop End")

    return Get_Comp

def __Naver_CompInfo(Dart_Corp_Code, Dart_Exp_Code, File_Name) :

    print(File_Name)
    if platform == 'darwin' :
        output = open(File_Name, 'a+t')
    else :
        output = open(File_Name, 'a+t', encoding='utf-8')
        
    print("############################################################")
    output.write("############################################################\n")
    print("# Dart_Naver_CompInfo Start")
    output.write("# Dart_Naver_CompInfo Start\n")
    print("############################################################")
    output.write("############################################################\n")
    print("")
    output.write("\n")
    url = "https://finance.naver.com/item/coinfo.nhn?code=" + Dart_Exp_Code
    output.write("%s \n" % url) # ---> 당시의 위치정보를 기반으로 15페이지 가져오는 듯, 어찌 가져오는지는 모르겠음...(2020.05.21)

    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) ' \
                             'AppleWebKit/537.11 (KHTML, like Gecko) ' \
                             'Chrome/23.0.1271.64 Safari/537.11', \
               'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8', \
               'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3', \
               'Accept-Encoding': 'none', \
               'Accept-Language': 'en-US,en;q=0.8', \
               'Connection': 'keep-alive'}
    req = Request(url=url, headers=headers)

    try :
        html = urlopen(req).read()
        # html = urlopen(url, context=context)
    except HTTPError as err:
        if err.code == 404 :
            print("# Error HTTP : 404")
            return
        elif err.code == 403 :
            print("# Error HTTP : 403")
            return
        elif err.code == 500 :
            print("# Error HTTP : 500")
            return
        elif err.code == 505:
            print("# Error HTTP : 505")
            return
        else:
            raise

    bs_object = bs4.BeautifulSoup(html, "html.parser")
    table = bs_object.find('div', {'id':'summary_info'})
    table_body = table.find_all('p')

    # print(table_body)
    for x in table_body :
        print(x.get_text())
        output.write("%s \n" % x.get_text())
    print("")

    output.write("\n")
    output.close()

def __Naver_BusinessType(Dart_Exp_Code, File_Name) :

    if platform == 'darwin' :
        output = open(File_Name, 'a+t')
    else :
        output = open(File_Name, 'a+t', encoding='utf-8')
        
    print("############################################################")
    output.write("############################################################\n")
    print("# 동종 업계 비교")
    output.write("# 동종 업계 비교\n")
    print("############################################################")
    output.write("############################################################\n")

    url = "https://finance.naver.com/item/main.nhn?code=" + Dart_Exp_Code
    output.write("%s \n" % url)

    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) ' \
                             'AppleWebKit/537.11 (KHTML, like Gecko) ' \
                             'Chrome/23.0.1271.64 Safari/537.11', \
               'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8', \
               'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3', \
               'Accept-Encoding': 'none', \
               'Accept-Language': 'en-US,en;q=0.8', \
               'Connection': 'keep-alive'}
    req = Request(url=url, headers=headers)

    try :
        html = urlopen(req).read()
        # html = urlopen(url, context=context)
    except HTTPError as err:
        if err.code == 404 :
            print("# Error HTTP : 404")
            return
        elif err.code == 403 :
            print("# Error HTTP : 403")
            return
        elif err.code == 500 :
            print("# Error HTTP : 500")
            return
        elif err.code == 505:
            print("# Error HTTP : 505")
            return
        else:
            raise

    bs_object = bs4.BeautifulSoup(html, "html.parser")
    table = bs_object.find('div', {'class':'section trade_compare'})
    table_data = pd.read_html(str(table))
    print(table_data)

    PBR_Raw = table_data[0]
    print(PBR_Raw.columns)
    print(PBR_Raw.values)

    output.write("%s\n" % table_data)
    output.write("\n")

    output.close()

def __Naver_Inst_Anal(Dart_Exp_Code) :

    # print("############################################################")
    # print("# Naver 투자 지표")
    # print("############################################################")

    url = "https://navercomp.wisereport.co.kr/v2/company/c1040001.aspx?cmp_cd=" + Dart_Exp_Code

    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) ' \
                             'AppleWebKit/537.11 (KHTML, like Gecko) ' \
                             'Chrome/23.0.1271.64 Safari/537.11', \
               'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8', \
               'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3', \
               'Accept-Encoding': 'none', \
               'Accept-Language': 'en-US,en;q=0.8', \
               'Connection': 'keep-alive'}
    req = Request(url=url, headers=headers)

    try :
        html = urlopen(req).read()
        # html = urlopen(url, context=context)
    except HTTPError as err:
        if err.code == 404 :
            print("# Error HTTP : 404")
            return
        elif err.code == 403 :
            print("# Error HTTP : 403")
            return
        elif err.code == 500 :
            print("# Error HTTP : 500")
            return
        elif err.code == 505:
            print("# Error HTTP : 505")
            return
        else:
            raise

    bs_object = bs4.BeautifulSoup(html, "html.parser")
    # print(bs_object)
    # table_anal = bs_object.find('table', {'class':'gHead01 all-width data-list'})
    # table_anal = bs_object.find('div', {'class':'tcRVArcVR1a2'})
    table_anal = bs_object.find('td', {'class':'cmp-table-cell td0301'})
    # print(table_anal.text) # ---> <class 'str'>

    str_anal = ""
    for find_text_list in table_anal.text :
        if "\n" not in find_text_list :
            # print(find_text_list, end="")
            str_anal += find_text_list
        else :
            str_anal += "\n"
    # print(str_anal.splitlines()) # ---> ['', '', 'EPS 3,014', 'BPS 14,640', 'PER 19.01', '업종PER N/A', 'PBR 3.91', '현금배당수익률 0.87%', '12월 결산', '']
    str_anal_DF = pd.DataFrame([x.split(';') for x in str_anal.split('\n')])
    str_anal_DF.dropna()

    string_PBR = str_anal_DF.iloc[6].values[0]
    # print(string_PBR)
    string_PBR = string_PBR.replace("PBR ","")
    # print(string_PBR)
    return string_PBR

def __Naver_Anal(Dart_Exp_Code, File_Name) :

    if platform == 'darwin' :
        output = open(File_Name, 'a+t')
    else :
        output = open(File_Name, 'a+t', encoding='utf-8')
        
    print("############################################################")
    output.write("############################################################\n")
    print("# 실적 동향")
    output.write("# 실적 동향\n")
    print("############################################################")
    output.write("############################################################\n")

    url = "https://finance.naver.com/item/main.nhn?code=" + Dart_Exp_Code
    output.write("%s \n" % url)

    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) ' \
                             'AppleWebKit/537.11 (KHTML, like Gecko) ' \
                             'Chrome/23.0.1271.64 Safari/537.11', \
               'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8', \
               'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3', \
               'Accept-Encoding': 'none', \
               'Accept-Language': 'en-US,en;q=0.8', \
               'Connection': 'keep-alive'}
    req = Request(url=url, headers=headers)

    try :
        html = urlopen(req).read()
        # html = urlopen(url, context=context)
    except HTTPError as err:
        if err.code == 404 :
            print("# Error HTTP : 404")
            return
        elif err.code == 403 :
            print("# Error HTTP : 403")
            return
        elif err.code == 500 :
            print("# Error HTTP : 500")
            return
        elif err.code == 505:
            print("# Error HTTP : 505")
            return
        else:
            raise

    bs_object = bs4.BeautifulSoup(html, "html.parser")
    table_anal = bs_object.find('table', {'class':'tb_type1 tb_num tb_type1_ifrs'})
    table_data_anal = pd.read_html(str(table_anal))

    table_anal_DF = table_data_anal[0]
    # print(table_anal_DF.columns)
    # MultiIndex([(  '주요재무정보',     '주요재무정보', '주요재무정보'),
    #         ('최근 연간 실적',    '2017.12', 'IFRS연결'),
    #         ('최근 연간 실적',    '2018.12', 'IFRS연결'),
    #         ('최근 연간 실적',    '2019.12', 'IFRS연결'),
    #         ('최근 연간 실적', '2020.12(E)', 'IFRS연결'),
    #         ('최근 분기 실적',    '2019.09', 'IFRS연결'),
    #         ('최근 분기 실적',    '2019.12', 'IFRS연결'),
    #         ('최근 분기 실적',    '2020.03', 'IFRS연결'),
    #         ('최근 분기 실적',    '2020.06', 'IFRS연결'),
    #         ('최근 분기 실적',    '2020.09', 'IFRS연결'),
    #         ('최근 분기 실적', '2020.12(E)', 'IFRS연결')],
    #        )
    print("############################")
    print(table_anal_DF.columns)
    print("############################")
    print(table_anal_DF.values)
    print("############################")
    # print(tabulate(table_anal_DF),tablefmt='psql')
    print("############################")

    output.write("%s\n" % table_anal_DF)
    output.write("%s\n" % tabulate(table_anal_DF))
    # f.write(tabulate(Write_TEXT))
    output.write("\n")

    output.close()

def __Naver_Per_Info(Dart_Exp_Code) :

    # print(File_Name)
    # output = open(File_Name, 'a+t', encoding='utf-8')
    # print("############################################################")
    # output.write("############################################################\n")
    # print("# Dart_Naver_CompInfo Start")
    # output.write("# Dart_Naver_CompInfo Start\n")
    # print("############################################################")
    # output.write("############################################################\n")
    # print("")
    # output.write("\n")
    url = "https://finance.naver.com/item/main.nhn?code="  + Dart_Exp_Code
    # output.write("%s \n" % url) # ---> 당시의 위치정보를 기반으로 15페이지 가져오는 듯, 어찌 가져오는지는 모르겠음...(2020.05.21)

    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) ' \
                             'AppleWebKit/537.11 (KHTML, like Gecko) ' \
                             'Chrome/23.0.1271.64 Safari/537.11', \
               'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8', \
               'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3', \
               'Accept-Encoding': 'none', \
               'Accept-Language': 'en-US,en;q=0.8', \
               'Connection': 'keep-alive'}
    req = Request(url=url, headers=headers)

    try :
        html = urlopen(req).read()
        # html = urlopen(url, context=context)
    except HTTPError as err:
        if err.code == 404 :
            print("# Error HTTP : 404")
            return
        elif err.code == 403 :
            print("# Error HTTP : 403")
            return
        elif err.code == 500 :
            print("# Error HTTP : 500")
            return
        elif err.code == 505:
            print("# Error HTTP : 505")
            return
        else:
            raise

    bs_object = bs4.BeautifulSoup(html, "html.parser")
    table = bs_object.find('table', attrs={'class':'per_table'})
    table_tr = bs_object.find_all('tr', attrs={'class':'strong'})

    data_1, data_2 = [], []

    for x in table_tr :
        test_x = x.text.replace("\t","")
        test_x = re.sub("\n\n+", " ", test_x)
        test_x = test_x.replace("\n","")
        if "동일업종" in test_x :
            # print("# for : %s " %  test_x)
            # print(int(filter(str.isdigit, test_x)))
            # data_1 = re.findall('\d+',test_x) # ---> - 값이 출력안됨
            data_1 = re.findall('-?\d+\.?\d*',test_x)

    rows_em = table.find_all('em')
    for x in rows_em :
        data_2.append(x.text)

    # print("# PER, EPS --> 최근4분기, cns_Per, cns_EPS ---> 추정, _PBR, BPS, 배당수익율")
    # print("# Data 1")
    # print(data_1)
    # data_Field_PER = data_1[0] + "." + data_1[1]
    # print(data_Field_PER)
    # print("# Data 2")
    # print(data_2)
    data_2.extend(data_1)

    return data_2

    # output.write("\n")
    # output.close()

def __Dart_Search_Code(Dart_Corp_Name, Dart_Exp_Code) :

    import dart_fss as dart
    from dart_fss import get_corp_list

    BEGIN_DATE = "20100101"

    # Open DART API KEY 설정
    dart.set_api_key(api_key=API_KEY)

    # DART 에 공시된 회사 리스트 불러오기
    corp_list = dart.get_corp_list()

    print("# Dart_Corp_Name : %s" % Dart_Corp_Name)

    Dart_DF = corp_list.find_by_corp_name(Dart_Corp_Name, exactly=True)[0]
    # Dart_DF = corp_list.find_by_corp_name('삼성전자', exactly=True)[0]
    # Dart_DF = corp_list.find_by_corp_name('삼성') # 삼성을 포함한 회사정보
    # Dart_DF = corp_list.find_by_stock_code('005930') # Stock 코드로 찾기
    # Dart_DF = corp_list.find_by_corp_code('00126380') # 다트에서 사용하는 회사코드를 이용한 찾기
    # print("# Dart_DF")
    # print(Dart_DF)
    print("")

def __Dart_Search_List(Dart_Corp_Code, Dart_Exp_Code, File_Name) :

    if platform == 'darwin' :
        output = open(File_Name, 'a+t')
    else :
        output = open(File_Name, 'a+t', encoding='utf-8')
        
    print("############################################################")
    output.write("############################################################\n")
    print("# 공시보고 전체 리스트")
    output.write("# 공시보고 전체 리스트\n")
    print("############################################################")
    output.write("############################################################\n")

    BEGIN_Y_DATE = "20100101"
    TODATE_Y_DATE = datetime.datetime.today().strftime("%Y%m%d")

    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) ' \
                                'AppleWebKit/537.11 (KHTML, like Gecko) ' \
                                'Chrome/23.0.1271.64 Safari/537.11', \
                'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8', \
                'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3', \
                'Accept-Encoding': 'none', \
                'Accept-Language': 'en-US,en;q=0.8', \
                'Connection': 'keep-alive'}

    # 공시 리스트 가져오기
    url = "http://opendart.fss.or.kr/api/list.json?crtfc_key=0fcb8ef56cf8e8b996d47937a8fea5aba8c8d1be&corp_code=" \
         + Dart_Corp_Code + "&bgn_de=" + BEGIN_Y_DATE + "&end_de=" + TODATE_Y_DATE + "&pblntf_ty=A&page_count=100"
    output.write("%s \n" % url)
    req = Request(url=url, headers=headers)
    html = urlopen(req).read()
    html_output = html.decode('utf-8')
    # print("# html_output")
    html_json = pd.read_json(html_output, encoding="utf-8")
    html_json = html_json.list.apply(pd.Series)
    # print(html_json) # corp_code, corp_name, stock_code, corp_cls, report_nm, rcept_no, flr_nm, rcept_dt, rm
    # print(type(html_json)) # <class 'pandas.core.frame.DataFrame'>
    print(html_json[['report_nm', 'rcept_no', 'rcept_dt']])
    output.write("%s \n" % html_json[['report_nm', 'rcept_no', 'rcept_dt']])

    html_json['report_type'] = ""
    html_json['report_type'][html_json['report_nm'].str.contains("사업보고서")] = "11011"
    html_json['report_type'][html_json['report_nm'].str.contains("반기보고서")] = "11012"
    html_json['report_type'][html_json['report_nm'].str.contains("분기보고서")] = "11013"
    # 사업보고서 : 11011, 반기보고서 : 11012, 1분기보고서 : 11013, 3분기보고서 : 11014 

    html_Y_report = html_json[html_json['report_type'].str.match('11011')]
    html_H_report = html_json[html_json['report_type'].str.match('11012')]

    print("# 사업보고서 리스트")
    print(html_Y_report)
    print("# 반기보고서 리스트")
    print(html_H_report)

    # print("# 사업보고서만 출력 1")
    # print(html_json[html_json['report_nm'].str.contains('사업보고서')])
    print("# 사업보고서, 분기보고서만 출력 2")
    output.write("# 사업보고서, 분기보고서만 출력 2\n")
    print(html_json[html_json['report_nm'].str.contains('사업보고서|반기보고서|분기보고서')])
    # print(html_json[html_json['report_nm'].str.contains('사업보고서|반기보고서', na=False)])
    output.write("%s \n" % html_json[html_json['report_nm'].str.contains('사업보고서|반기보고서')])

    # html_json = json.dumps(html_json, ensure_ascii=False).encode('utf8')
    # html_json = pd.json_normalize(html_json,'list')
    # html_json = pd.json_normalize(html_json,'list', ['account_nm'],['thstrm_nm'],['thstrm_dt'],['thstrm_amount'],['frmtrm_nm'])
    # print(html_json)
    # py_json = son.loads(json_file.decode('utf-8'))
    # python_df = pd_dataframe(py_json["key"][0]["key2"], columns = ['col1', 'col2','col3','col4'])kk
    print("")
    output.write("\n")

    print("return 값은 : %s" % html_json[['rcept_no']].head(1))
    output.write("return 값은 : %s" % html_json[['rcept_no']].head(1))

    output.close()

    return html_json[['rcept_no']].head(1)

def __Dart_Search_1st(Dart_Corp_Code, Dart_Exp_Code, File_Name) :

    if platform == 'darwin' :
        output = open(File_Name, 'a+t')
    else :
        output = open(File_Name, 'a+t', encoding='utf-8')
        
    print("############################################################")
    output.write("############################################################\n")
    print("# 최대주주 현황")
    output.write("# 최대주주 현황\n")
    print("############################################################")
    output.write("############################################################\n")

    BEGIN_Y_DATE = int("2015")
    TODATE_Y_DATE = int(datetime.datetime.today().strftime("%Y"))+1

    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) ' \
                                'AppleWebKit/537.11 (KHTML, like Gecko) ' \
                                'Chrome/23.0.1271.64 Safari/537.11', \
                'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8', \
                'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3', \
                'Accept-Encoding': 'none', \
                'Accept-Language': 'en-US,en;q=0.8', \
                'Connection': 'keep-alive'}

    for x in range(BEGIN_Y_DATE, TODATE_Y_DATE) :
        url = "http://opendart.fss.or.kr/api/hyslrSttus.json?crtfc_key=0fcb8ef56cf8e8b996d47937a8fea5aba8c8d1be&corp_code=" \
            + Dart_Corp_Code + "&bsns_year=" + str(x) + "&reprt_code=11011"
        # 1분기보고서 : 11013 반기보고서 : 11012 3분기보고서 : 11014 사업보고서 : 11011
        output.write("%s \n" % url)
        req = Request(url=url, headers=headers)
        html = urlopen(req).read()
        html_output = html.decode('utf-8')
        print("# %s년 최대주주 현황" % x)
        output.write("# %s년 최대주주 현황\n" % x)
        if "013" in html_output : # {"status":"013","message":"조회된 데이타가 없습니다."}
            print("# ---> 조회 내역이 없음")
            output.write("# ---> 조회 내역이 없음\n")
            continue
        html_json = pd.read_json(html_output, encoding="utf-8")
        html_json = html_json.list.apply(pd.Series)
        print("# 접수번호, 성명, 관계, 주식종류, 기초 소유 주식 지분율, 기말 소유 주식 지분율, 비고")
        print(html_json[['rcept_no', 'nm', 'relate', 'stock_knd', 'bsis_posesn_stock_qota_rt', 'trmend_posesn_stock_qota_rt', 'rm']])
        output.write("%s \n" % html_json[['rcept_no', 'nm', 'relate', 'stock_knd', 'bsis_posesn_stock_qota_rt', 'trmend_posesn_stock_qota_rt', 'rm']])
        print("")

    output.write("\n")
    output.close()

def __Dart_Search_1st_Diff(Dart_Corp_Code, Dart_Exp_Code, File_Name) :

    if platform == 'darwin' :
        output = open(File_Name, 'a+t')
    else :
        output = open(File_Name, 'a+t', encoding='utf-8')
        
    print("############################################################")
    output.write("############################################################\n")
    print("# 최대 주주 변동 내역")
    output.write("# 최대 주주 변동 내역\n")
    print("############################################################")
    output.write("############################################################\n")

    BEGIN_Y_DATE = int("2015")
    TODATE_Y_DATE = int(datetime.datetime.today().strftime("%Y"))+1

    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) ' \
                                'AppleWebKit/537.11 (KHTML, like Gecko) ' \
                                'Chrome/23.0.1271.64 Safari/537.11', \
                'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8', \
                'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3', \
                'Accept-Encoding': 'none', \
                'Accept-Language': 'en-US,en;q=0.8', \
                'Connection': 'keep-alive'}

    for x in range(BEGIN_Y_DATE, TODATE_Y_DATE) :
        url = "http://opendart.fss.or.kr/api/hyslrChgSttus.json?crtfc_key=0fcb8ef56cf8e8b996d47937a8fea5aba8c8d1be&corp_code=" \
            + Dart_Corp_Code + "&bsns_year=" + str(x) + "&reprt_code=11011"
        output.write("%s \n" % url)
        req = Request(url=url, headers=headers)
        html = urlopen(req).read()
        html_output = html.decode('utf-8')
        print("# %s년 최대주주 변동 현황" % x)
        output.write("# %s년 최대주주 변동 현황\n" % x)
        if "013" in html_output : # {"status":"013","message":"조회된 데이타가 없습니다."}
            print("# ---> 조회 내역이 없음")
            output.write("# ---> 조회 내역이 없음\n")
            continue
        html_json = pd.read_json(html_output, encoding="utf-8")
        html_json = html_json.list.apply(pd.Series)
        print("# 접수번호, 변동일, 최대주주명, 지분율, 변동원인, 비고")
        output.write("# 접수번호, 변동일, 최대주주명, 지분율, 변동원인, 비고\n")
        print(html_json[['rcept_no', 'change_on', 'mxmm_shrholdr_nm', 'qota_rt', 'change_cause', 'rm']])
        output.write("%s \n" % html_json[['rcept_no', 'change_on', 'mxmm_shrholdr_nm', 'qota_rt', 'change_cause', 'rm']])
        print("")

    output.write("\n")
    output.close()

def __Dart_Search_Stock(Dart_Corp_Code, Dart_Exp_Code, File_Name) :

    if platform == 'darwin' :
        output = open(File_Name, 'a+t')
    else :
        output = open(File_Name, 'a+t', encoding='utf-8')

    print("############################################################")
    output.write("############################################################\n")
    print("# 자기주식 취득 및 처분 현황")
    output.write("# 자기주식 취득 및 처분 현황\n")
    print("############################################################")
    output.write("############################################################\n")

    BEGIN_Y_DATE = int("2015")
    TODATE_Y_DATE = int(datetime.datetime.today().strftime("%Y"))+1

    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) ' \
                                'AppleWebKit/537.11 (KHTML, like Gecko) ' \
                                'Chrome/23.0.1271.64 Safari/537.11', \
                'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8', \
                'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3', \
                'Accept-Encoding': 'none', \
                'Accept-Language': 'en-US,en;q=0.8', \
                'Connection': 'keep-alive'}

    for x in range(BEGIN_Y_DATE, TODATE_Y_DATE) :
        url = "http://opendart.fss.or.kr/api/tesstkAcqsDspsSttus.json?crtfc_key=0fcb8ef56cf8e8b996d47937a8fea5aba8c8d1be&corp_code=" \
            + Dart_Corp_Code + "&bsns_year=" + str(x) + "&reprt_code=11011"
        output.write("%s \n" % url)
        req = Request(url=url, headers=headers)
        html = urlopen(req).read()
        html_output = html.decode('utf-8')
        print("# %s년 자기주식 취득 / 처분 현황" % x)
        output.write("# %s년 자기주식 취득 / 처분 현황\n" % x)
        if "013" in html_output : # {"status":"013","message":"조회된 데이타가 없습니다."}
            print("# ---> 조회 내역이 없음")
            output.write("# ---> 조회 내역이 없음\n")
            continue
        html_json = pd.read_json(html_output, encoding="utf-8")
        html_json = html_json.list.apply(pd.Series)
        print("# 접수번호, 취득방법 대분류, 취득방법 중분류, 취득방법 소분류, 주식 종류, 기초 수량, 변동 수량 취득, 변동 수량 처분, 변동 수량 소각, 기말 수량, 비고")
        output.write("# 접수번호, 취득방법 대분류, 취득방법 중분류, 취득방법 소분류, 주식 종류, 기초 수량, 변동 수량 취득, 변동 수량 처분, 변동 수량 소각, 기말 수량, 비고\n")
        print(html_json[['rcept_no', 'acqs_mth1', 'acqs_mth2', 'acqs_mth3', 'stock_knd', 'bsis_qy', 'change_qy_acqs', 'change_qy_dsps', 'change_qy_incnr', 'rm']])
        print("%s \n" % html_json[['rcept_no', 'acqs_mth1', 'acqs_mth2', 'acqs_mth3', 'stock_knd', 'bsis_qy', 'change_qy_acqs', 'change_qy_dsps', 'change_qy_incnr', 'rm']])
        print("")

    output.write("\n")
    output.close()

def __Dart_Search_Emp(Dart_Corp_Code, Dart_Exp_Code, File_Name) :

    if platform == 'darwin' :
        output = open(File_Name, 'a+t')
    else :
        output = open(File_Name, 'a+t', encoding='utf-8')

    print("############################################################")
    output.write("############################################################\n")
    print("# 직원 현황")
    output.write("# 직원 현황\n")
    print("############################################################")
    output.write("############################################################\n")

    BEGIN_Y_DATE = int("2015")
    TODATE_Y_DATE = int(datetime.datetime.today().strftime("%Y"))+1

    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) ' \
                                'AppleWebKit/537.11 (KHTML, like Gecko) ' \
                                'Chrome/23.0.1271.64 Safari/537.11', \
                'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8', \
                'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3', \
                'Accept-Encoding': 'none', \
                'Accept-Language': 'en-US,en;q=0.8', \
                'Connection': 'keep-alive'}

    for x in range(BEGIN_Y_DATE, TODATE_Y_DATE) :
        url = "http://opendart.fss.or.kr/api/empSttus.json?crtfc_key=0fcb8ef56cf8e8b996d47937a8fea5aba8c8d1be&corp_code=" \
            + Dart_Corp_Code + "&bsns_year=" + str(x) + "&reprt_code=11011"
        print(url)
        output.write("%s \n" % url)
        req = Request(url=url, headers=headers)
        html = urlopen(req).read()
        html_output = html.decode('utf-8')
        print("# %s년 직원현황" % x)
        output.write("# %s년 직원현황\n" % x)
        if "013" in html_output : # {"status":"013","message":"조회된 데이타가 없습니다."}
            print("# ---> 조회 내역이 없음")
            output.write("# ---> 조회 내역이 없음\n")
            continue
        html_json = pd.read_json(html_output, encoding="utf-8")
        html_json = html_json.list.apply(pd.Series)
        print("# 접수번호, 성별, 사업부문, 정규직 수, 계약직 수, 합계, 평균근속연수, 1인평균 급여, 비고")
        output.write("# 접수번호, 성별, 사업부문, 정규직 수, 계약직 수, 합계, 평균근속연수, 1인평균 급여, 비고\n")
        print(html_json[['rcept_no', 'sexdstn', 'fo_bbm', 'rgllbr_co', 'cnttk_co', 'sm', 'avrg_cnwk_sdytrn', 'jan_salary_am', 'rm']])
        output.write("%s \n" % html_json[['rcept_no', 'sexdstn', 'fo_bbm', 'rgllbr_co', 'cnttk_co', 'sm', 'avrg_cnwk_sdytrn', 'jan_salary_am', 'rm']])
        print("")

    output.write("\n")
    output.close()

def __Dart_Search_Report(Dart_Corp_Code, Dart_Exp_Code, File_Name) :

    if platform == 'darwin' :
        output = open(File_Name, 'a+t')
    else :
        output = open(File_Name, 'a+t', encoding='utf-8')

    print("############################################################")
    output.write("############################################################\n")
    print("# 재무제표 확인 : 단일회사 전체 재무제표 개발가이드")
    output.write("# 재무제표 확인 : 단일회사 전체 재무제표 개발가이드\n")
    print("############################################################")
    output.write("############################################################\n")
 
    BEGIN_Y_DATE = int("2015")
    TODATE_Y_DATE = int(datetime.datetime.today().strftime("%Y"))+1

    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) ' \
                                'AppleWebKit/537.11 (KHTML, like Gecko) ' \
                                'Chrome/23.0.1271.64 Safari/537.11', \
                'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8', \
                'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3', \
                'Accept-Encoding': 'none', \
                'Accept-Language': 'en-US,en;q=0.8', \
                'Connection': 'keep-alive'}

    # 재무정보 가져오기
    for x in range(BEGIN_Y_DATE, TODATE_Y_DATE) :
        url = "https://opendart.fss.or.kr/api/fnlttSinglAcnt.json?crtfc_key=0fcb8ef56cf8e8b996d47937a8fea5aba8c8d1be&corp_code=" \
        + Dart_Corp_Code + "&bsns_year=" + str(x) + "&reprt_code=11011"
        print(url)
        output.write("%s \n" % url)
        req = Request(url=url, headers=headers)
        html = urlopen(req).read()
        html_output = html.decode('utf-8')
        print("############################################################")
        output.write("############################################################\n")
        print("# %s년 재무재표......." % x)
        output.write("# %s년 재무재표.......\n" % x)
        if "조회된" in html_output : # {"status":"013","message":"조회된 데이타가 없습니다."}
            print("# ---> 조회 내역이 없음")
            output.write("# ---> 조회 내역이 없음\n")
            continue
        print(html_output) # ---> 재무제표 출력

        html_json = pd.read_json(html_output, encoding="utf-8")
        html_json = html_json.list.apply(pd.Series)
        # print(html_json)
        # print(html_json[['reprt_code','bsns_year', 'fs_div', 'fs_nm', 'sj_div', 'sj_nm', 'account_nm', 'thstrm_dt', 'thstrm_amount', 'frmtrm_dt', 'frmtrm_amount', 'bfefrmtrm_dt', 'bfefrmtrm_amount', 'ord']])
        # print(html_json[['bsns_year', 'fs_nm', 'sj_nm', 'account_nm', 'thstrm_dt', 'thstrm_amount', 'frmtrm_dt', 'frmtrm_amount', 'bfefrmtrm_dt', 'bfefrmtrm_amount', 'ord']])

        # print("############################################################")
        # print(html_json.columns)
        # print(html_json.values)
        # print("############################################################")
        # print(html_json[html_json['account_nm'].str.match('매출액')])
        # print("")
        # print(html_json[html_json['account_nm'].str.match('영업이익')])
        # print("")
        # print(html_json[html_json['account_nm'].str.match('법인세차감전 순이익')])
        # print("")
        # print(html_json[html_json['account_nm'].str.match('당기순이익')])
        # print("")

        print("############################################################")
        output.write("############################################################\n")
        print("# 재무재표 정제해서 출력.......")
        output.write("# 재무재표 정제해서 출력.......\n")
        print("############################################################")
        output.write("############################################################\n")
        if 'bfefrmtrm_dt' not in html_json : # bfefrmtrm_dt 가 없는 경우도 있다.
            print("# 마지막 3기가 없음")
            output.write("# 마지막 3기가 없음\n")
            print(html_json.loc[html_json['account_nm']=='영업이익', ['account_nm', 'thstrm_dt', 'thstrm_amount','frmtrm_dt', 'frmtrm_amount']])
            output.write("%s\n" % html_json.loc[html_json['account_nm']=='영업이익', ['account_nm', 'thstrm_dt', 'thstrm_amount','frmtrm_dt', 'frmtrm_amount']])
            print(html_json.loc[html_json['account_nm']=='법인세차감전 순이익', ['account_nm', 'thstrm_dt', 'thstrm_amount','frmtrm_dt', 'frmtrm_amount']])
            output.write("%s\n" % html_json.loc[html_json['account_nm']=='법인세차감전 순이익', ['account_nm', 'thstrm_dt', 'thstrm_amount','frmtrm_dt', 'frmtrm_amount']])
            print(html_json.loc[html_json['account_nm']=='당기순이익', ['account_nm', 'thstrm_dt', 'thstrm_amount','frmtrm_dt', 'frmtrm_amount']])
            output.write("%s\n" % html_json.loc[html_json['account_nm']=='당기순이익', ['account_nm', 'thstrm_dt', 'thstrm_amount','frmtrm_dt', 'frmtrm_amount']])
        elif 'frmtrm_dt' not in html_json :
            print("# 마지막 2기가 없음")
            output.write("# 마지막 2기가 없음\n")
            print(html_json.loc[html_json['account_nm']=='영업이익', ['account_nm', 'thstrm_dt', 'thstrm_amount']])
            output.write("%s\n" % html_json.loc[html_json['account_nm']=='영업이익', ['account_nm', 'thstrm_dt', 'thstrm_amount']])
            print(html_json.loc[html_json['account_nm']=='법인세차감전 순이익', ['account_nm', 'thstrm_dt', 'thstrm_amount']])
            output.write("%s\n" % html_json.loc[html_json['account_nm']=='법인세차감전 순이익', ['account_nm', 'thstrm_dt', 'thstrm_amount']])
            print(html_json.loc[html_json['account_nm']=='당기순이익', ['account_nm', 'thstrm_dt', 'thstrm_amount']])
            output.write("%s\n" % html_json.loc[html_json['account_nm']=='당기순이익', ['account_nm', 'thstrm_dt', 'thstrm_amount']])
        else :
            print(html_json.loc[html_json['account_nm']=='영업이익', ['account_nm', 'thstrm_dt', 'thstrm_amount','frmtrm_dt', 'frmtrm_amount', 'bfefrmtrm_dt', 'bfefrmtrm_amount']])
            output.write("%s\n" % html_json.loc[html_json['account_nm']=='영업이익', ['account_nm', 'thstrm_dt', 'thstrm_amount','frmtrm_dt', 'frmtrm_amount', 'bfefrmtrm_dt', 'bfefrmtrm_amount']])
            print(html_json.loc[html_json['account_nm']=='법인세차감전 순이익', ['account_nm', 'thstrm_dt', 'thstrm_amount','frmtrm_dt', 'frmtrm_amount', 'bfefrmtrm_dt', 'bfefrmtrm_amount']])
            output.write("%s\n" % html_json.loc[html_json['account_nm']=='법인세차감전 순이익', ['account_nm', 'thstrm_dt', 'thstrm_amount','frmtrm_dt', 'frmtrm_amount', 'bfefrmtrm_dt', 'bfefrmtrm_amount']])
            print(html_json.loc[html_json['account_nm']=='당기순이익', ['account_nm', 'thstrm_dt', 'thstrm_amount','frmtrm_dt', 'frmtrm_amount', 'bfefrmtrm_dt', 'bfefrmtrm_amount']])
            output.write("%s\n" % html_json.loc[html_json['account_nm']=='당기순이익', ['account_nm', 'thstrm_dt', 'thstrm_amount','frmtrm_dt', 'frmtrm_amount', 'bfefrmtrm_dt', 'bfefrmtrm_amount']])

    TODATE_Y_DATE = int(datetime.datetime.today().strftime("%Y"))
    TODATE_Y_DATE = int(datetime.datetime.today().strftime("%Y"))-1
    print(int(datetime.datetime.today().strftime("%Y")))
    print(int(datetime.datetime.today().strftime("%Y"))-1)
    print("############################################################")
    output.write("############################################################\n")
    print("# 마지막만 정제 ---> %s .......1" % TODATE_Y_DATE)
    output.write("# 마지막만 정제 ---> %s .......1\n" % TODATE_Y_DATE)
    print("############################################################")
    output.write("############################################################\n")
    url = "https://opendart.fss.or.kr/api/fnlttSinglAcnt.json?crtfc_key=0fcb8ef56cf8e8b996d47937a8fea5aba8c8d1be&corp_code=" \
        + Dart_Corp_Code + "&bsns_year=" + str(TODATE_Y_DATE) + "&reprt_code=11011"
    print(url)
    output.write("%s \n" % url)
    req = Request(url=url, headers=headers)
    html = urlopen(req).read()
    html_output = html.decode('utf-8')
    if "조회된" in html_output : # {"status":"013","message":"조회된 데이타가 없습니다."}
        TODATE_Y_DATE = int(datetime.datetime.today().strftime("%Y"))-2
        print("# ---> 조회 내역이 없음 ---> 그 이전 년도 조회 : %s" % TODATE_Y_DATE)
        output.write("# ---> 조회 내역이 없음 ---> 그 이전 년도 조회 : %s\n" % TODATE_Y_DATE)
        url = "https://opendart.fss.or.kr/api/fnlttSinglAcnt.json?crtfc_key=0fcb8ef56cf8e8b996d47937a8fea5aba8c8d1be&corp_code=" \
            + Dart_Corp_Code + "&bsns_year=" + str(TODATE_Y_DATE) + "&reprt_code=11011"
        print(url)
        output.write("%s \n" % url)
        req = Request(url=url, headers=headers)
        html = urlopen(req).read()
        html_output = html.decode('utf-8')

    html_json = pd.read_json(html_output, encoding="utf-8")
    html_json = html_json.list.apply(pd.Series)
    print("############################################################")
    output.write("############################################################\n")
    print("# 마지막만 정제.......2 ---> 요약 연결재무정보 ")
    output.write("# 마지막만 정제.......2 ---> 요약 연결재무정보 \n")
    print("############################################################")
    output.write("############################################################\n")

    print("# 매출액")
    output.write("# 매출액\n")
    print("############################################################\n")
    output.write("############################################################\n")
    sales_1_name = html_json.loc[html_json['account_nm']=='매출액', ['thstrm_nm']].values[0]
    sales_1_date = html_json.loc[html_json['account_nm']=='매출액', ['thstrm_dt']].values[0]
    sales_1_value = html_json.loc[html_json['account_nm']=='매출액', ['thstrm_amount']].values[0]
    sales_2_name = html_json.loc[html_json['account_nm']=='매출액', ['frmtrm_nm']].values[0]
    sales_2_date = html_json.loc[html_json['account_nm']=='매출액', ['frmtrm_dt']].values[0]
    sales_2_value = html_json.loc[html_json['account_nm']=='매출액', ['frmtrm_amount']].values[0]
    sales_3_name = html_json.loc[html_json['account_nm']=='매출액', ['bfefrmtrm_nm']].values[0]
    sales_3_date = html_json.loc[html_json['account_nm']=='매출액', ['bfefrmtrm_dt']].values[0]
    sales_3_value = html_json.loc[html_json['account_nm']=='매출액', ['bfefrmtrm_amount']].values[0]

    # print(type(sales_1_name)) # ---> <class 'numpy.ndarray'>
    # print(type(sales_1_name.tostring())) # ---> <class 'bytes'>
    # print(type(int.from_bytes(sales_1_name.tostring(), byteorder='big'))) # ---> <class 'bytes'>
    # print(type(int.from_bytes(sales_1_name.tostring(), byteorder='little'))) # ---> <class 'bytes'>
    # test_value1 = int.from_bytes(sales_1_name.tostring(), byteorder='big')
    # test_value2 = int.from_bytes(sales_1_name.tostring(), byteorder='little')
    # print(test_value1/8192)
    # print(test_value2)

    # print(type(sales_1_value))
    # print("############################################################")

    print("%s - %s : %s " % (sales_1_name, sales_1_date, sales_1_value))
    output.write("%s - %s : %s \n" % (sales_1_name, sales_1_date, sales_1_value))
    print("%s - %s : %s " % (sales_2_name, sales_2_date, sales_2_value))
    output.write("%s - %s : %s \n" % (sales_2_name, sales_2_date, sales_2_value))
    print("%s - %s : %s " % (sales_3_name, sales_3_date, sales_3_value))
    output.write("%s - %s : %s \n" % (sales_3_name, sales_3_date, sales_3_value))
    print("")
    output.write("\n")
    sales_name_list = list(sales_1_name)
    sales_name_list.append(sales_2_name)
    sales_name_list.append(sales_3_name)
    sales_value_list = list(sales_1_value)
    sales_value_list.append(sales_2_value)
    sales_value_list.append(sales_3_value)
    # print("############################################################")
    # print(sales_name_list)
    # print(sales_value_list)
    # print("############################################################")
    # plt.plot(sales_name_list, sales_value_list)	# line 그래프를 그립니다
    print("############################################################")
    output.write("############################################################\n")
    # plt.show()	# 그래프를 화면에 보여줍니다
 
    print("# 영업이익")
    output.write("# 영업이익\n")
    print("############################################################")
    output.write("############################################################\n")
    profit_1_name = html_json.loc[html_json['account_nm']=='영업이익', ['thstrm_nm']].values[0]
    profit_1_date = html_json.loc[html_json['account_nm']=='영업이익', ['thstrm_dt']].values[0]
    profit_1_value = html_json.loc[html_json['account_nm']=='영업이익', ['thstrm_amount']].values[0]
    profit_2_name = html_json.loc[html_json['account_nm']=='영업이익', ['frmtrm_nm']].values[0]
    profit_2_date = html_json.loc[html_json['account_nm']=='영업이익', ['frmtrm_dt']].values[0]
    profit_2_value = html_json.loc[html_json['account_nm']=='영업이익', ['frmtrm_amount']].values[0]
    profit_3_name = html_json.loc[html_json['account_nm']=='영업이익', ['bfefrmtrm_nm']].values[0]
    profit_3_date = html_json.loc[html_json['account_nm']=='영업이익', ['bfefrmtrm_dt']].values[0]
    profit_3_value = html_json.loc[html_json['account_nm']=='영업이익', ['bfefrmtrm_amount']].values[0]

    print("%s - %s : %s" % (profit_1_name, profit_1_date, profit_1_value))
    output.write("%s - %s : %s\n" % (profit_1_name, profit_1_date, profit_1_value))
    print("%s - %s : %s" % (profit_2_name, profit_2_date, profit_2_value))
    output.write("%s - %s : %s\n" % (profit_2_name, profit_2_date, profit_2_value))
    print("%s - %s : %s" % (profit_3_name, profit_3_date, profit_3_value))
    output.write("%s - %s : %s\n" % (profit_3_name, profit_3_date, profit_3_value))
    print("")
    output.write("\n")

    CorporateTax_1_name = html_json.loc[html_json['account_nm']=='법인세차감전 순이익', ['thstrm_nm']].values[0]
    CorporateTax_1_date = html_json.loc[html_json['account_nm']=='법인세차감전 순이익', ['thstrm_dt']].values[0]
    CorporateTax_1_value = html_json.loc[html_json['account_nm']=='법인세차감전 순이익', ['thstrm_amount']].values[0]
    CorporateTax_2_name = html_json.loc[html_json['account_nm']=='법인세차감전 순이익', ['frmtrm_nm']].values[0]
    CorporateTax_2_date = html_json.loc[html_json['account_nm']=='법인세차감전 순이익', ['frmtrm_dt']].values[0]
    CorporateTax_2_value = html_json.loc[html_json['account_nm']=='법인세차감전 순이익', ['frmtrm_amount']].values[0]
    CorporateTax_3_name = html_json.loc[html_json['account_nm']=='법인세차감전 순이익', ['bfefrmtrm_nm']].values[0]
    CorporateTax_3_date = html_json.loc[html_json['account_nm']=='법인세차감전 순이익', ['bfefrmtrm_dt']].values[0]
    CorporateTax_3_value = html_json.loc[html_json['account_nm']=='법인세차감전 순이익', ['bfefrmtrm_amount']].values[0]

    print("# 법인세차감전 순이익")
    output.write("# 법인세차감전 순이익\n")
    print("############################################################")
    output.write("############################################################\n")
    print("%s - %s : %s " % (CorporateTax_1_name, CorporateTax_1_date, CorporateTax_1_value))
    output.write("%s - %s : %s\n" % (CorporateTax_1_name, CorporateTax_1_date, CorporateTax_1_value))
    print("%s - %s : %s " % (CorporateTax_2_name, CorporateTax_2_date, CorporateTax_2_value))
    output.write("%s - %s : %s\n" % (CorporateTax_2_name, CorporateTax_2_date, CorporateTax_2_value))
    print("%s - %s : %s " % (CorporateTax_3_name, CorporateTax_3_date, CorporateTax_3_value))
    output.write("%s - %s : %s\n" % (CorporateTax_3_name, CorporateTax_3_date, CorporateTax_3_value))
    print("")
    output.write("\n")

    NetIncome_tax_1_name = html_json.loc[html_json['account_nm']=='당기순이익', ['thstrm_nm']].values[0]
    NetIncome_tax_1_date = html_json.loc[html_json['account_nm']=='당기순이익', ['thstrm_dt']].values[0]
    NetIncome_tax_1_value = html_json.loc[html_json['account_nm']=='당기순이익', ['thstrm_amount']].values[0]
    NetIncome_tax_2_name = html_json.loc[html_json['account_nm']=='당기순이익', ['frmtrm_nm']].values[0]
    NetIncome_tax_2_date = html_json.loc[html_json['account_nm']=='당기순이익', ['frmtrm_dt']].values[0]
    NetIncome_tax_2_value = html_json.loc[html_json['account_nm']=='당기순이익', ['frmtrm_amount']].values[0]
    NetIncome_tax_3_name = html_json.loc[html_json['account_nm']=='당기순이익', ['bfefrmtrm_nm']].values[0]
    NetIncome_tax_3_date = html_json.loc[html_json['account_nm']=='당기순이익', ['bfefrmtrm_dt']].values[0]
    NetIncome_tax_3_value = html_json.loc[html_json['account_nm']=='당기순이익', ['bfefrmtrm_amount']].values[0]

    print("# 당기 순이익")
    output.write("# 당기 순이익\n")
    print("############################################################")
    output.write("############################################################\n")
    print("%s - %s : %s " % (NetIncome_tax_1_name, NetIncome_tax_1_date, NetIncome_tax_1_value))
    output.write("%s - %s : %s\n" % (NetIncome_tax_1_name, NetIncome_tax_1_date, NetIncome_tax_1_value))
    print("%s - %s : %s " % (NetIncome_tax_2_name, NetIncome_tax_2_date, NetIncome_tax_2_value))
    output.write("%s - %s : %s\n" % (NetIncome_tax_2_name, NetIncome_tax_2_date, NetIncome_tax_2_value))
    print("%s - %s : %s " % (NetIncome_tax_3_name, NetIncome_tax_3_date, NetIncome_tax_3_value))
    output.write("%s - %s : %s\n" % (NetIncome_tax_3_name, NetIncome_tax_3_date, NetIncome_tax_3_value))
    print("")
    output.write("\n")

    output.write("\n")
    output.close()

def __Dart_Search_Report_Detail(Dart_Corp_Code, Dart_Exp_Code, File_Name) :

    BEGIN_Y_DATE = int("2015")
    TODATE_Y_DATE = int(datetime.datetime.today().strftime("%Y"))+1

    if platform == 'darwin' :
        output = open(File_Name, 'a+t')
    else :
        output = open(File_Name, 'a+t', encoding='utf-8')
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) ' \
                                'AppleWebKit/537.11 (KHTML, like Gecko) ' \
                                'Chrome/23.0.1271.64 Safari/537.11', \
                'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8', \
                'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3', \
                'Accept-Encoding': 'none', \
                'Accept-Language': 'en-US,en;q=0.8', \
                'Connection': 'keep-alive'}

    url = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json?crtfc_key=0fcb8ef56cf8e8b996d47937a8fea5aba8c8d1be&corp_code=" \
        + Dart_Corp_Code + "&bsns_year=" + str(TODATE_Y_DATE) + "&reprt_code=11011&fs_div=OFS"
    print(url)
    output.write("%s \n" % url)
    req = Request(url=url, headers=headers)
    html = urlopen(req).read()
    html_output = html.decode('utf-8')

    # print("# html_output 1")
    # print(html_output)

    if "조회된" in html_output : # {"status":"013","message":"조회된 데이타가 없습니다."}
        TODATE_Y_DATE = int(datetime.datetime.today().strftime("%Y"))-2
        print("# ---> 조회 내역이 없음 ---> 그 이전 년도 조회 : %s" % TODATE_Y_DATE)
        output.write("# ---> 조회 내역이 없음 ---> 그 이전 년도 조회 : %s\n" % TODATE_Y_DATE)
        url = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json?crtfc_key=0fcb8ef56cf8e8b996d47937a8fea5aba8c8d1be&corp_code=" \
            + Dart_Corp_Code + "&bsns_year=" + str(TODATE_Y_DATE) + "&reprt_code=11011&fs_div=OFS"
        print(url)
        output.write("%s \n" % url)
        req = Request(url=url, headers=headers)
        html = urlopen(req).read()
        html_output = html.decode('utf-8')
        # ---> 아래 또 없다면 종료
        if "조회된" in html_output : # {"status":"013","message":"조회된 데이타가 없습니다."}
            TODATE_Y_DATE = int(datetime.datetime.today().strftime("%Y"))-2
            print("# ---> 또 조회 내역이 없음 ---> 종료\n")
            output.write("# ---> 또 조회 내역이 없음 ---> 종료\n")
            return None

        # print("# html_output 2")
        # print(html_output)

    html_json = pd.read_json(html_output, encoding="utf-8")
    html_json = html_json.list.apply(pd.Series)

    print("########## html_json extract")
    # print(html_json.loc[html_json['sj_div']=='CF'])
    Extract_Data = html_json.loc[html_json['sj_div']=='CF']
    if 'frmtrm_nm' in Extract_Data.columns :
        print(Extract_Data[['account_nm', 'account_detail', 'thstrm_nm', 'thstrm_amount']])
    elif 'bfefrmtrm_nm' in Extract_Data.columns :
        print(Extract_Data[['account_nm', 'account_detail', 'thstrm_nm', 'thstrm_amount', 'frmtrm_nm', 'frmtrm_amount']])
    else :
        print(Extract_Data[['account_nm', 'account_detail', 'thstrm_nm', 'thstrm_amount', 'frmtrm_nm', 'frmtrm_amount', 'bfefrmtrm_nm', 'bfefrmtrm_amount']])

    # CashFlow_1_name = html_json.loc[html_json['account_nm']=='영업활동현금흐름', ['thstrm_nm']].values[0]
    # IndexError: index 0 is out of bounds for axis 0 with size 0
    if 'frmtrm_nm' in Extract_Data.columns and 'bfefrmtrm_nm' in Extract_Data.columns :
        CashFlow_1_name = html_json.loc[html_json['account_nm']=='영업활동현금흐름', ['thstrm_nm']].values[0]
        CashFlow_1_value = html_json.loc[html_json['account_nm']=='영업활동현금흐름', ['thstrm_amount']].values[0]
        CashFlow_2_name = html_json.loc[html_json['account_nm']=='영업활동현금흐름', ['frmtrm_nm']].values[0]
        CashFlow_2_value = html_json.loc[html_json['account_nm']=='영업활동현금흐름', ['frmtrm_amount']].values[0]
        CashFlow_3_name = html_json.loc[html_json['account_nm']=='영업활동현금흐름', ['bfefrmtrm_nm']].values[0]
        CashFlow_3_value = html_json.loc[html_json['account_nm']=='영업활동현금흐름', ['bfefrmtrm_amount']].values[0]

        print("############################################################")
        output.write("############################################################\n")
        print("# 영업활동현금흐름")
        output.write("# 영업활동현금흐름\n")
        print("############################################################")
        output.write("############################################################\n")
        print("%s : %s " % (CashFlow_1_name, CashFlow_1_value))
        output.write("%s : %s\n" % (CashFlow_1_name, CashFlow_1_value))
        print("%s : %s " % (CashFlow_2_name, CashFlow_2_value))
        output.write("%s : %s\n" % (CashFlow_2_name, CashFlow_2_value))
        print("%s : %s " % (CashFlow_3_name, CashFlow_3_value))
        output.write("%s : %s\n" % (CashFlow_3_name, CashFlow_3_value))
        print("")
        output.write("\n")

        # QuarterProfit_1_name = html_json.loc[html_json['account_nm']=='당기순이익(손실)', ['thstrm_nm']].values[0]
        # IndexError: index 0 is out of bounds for axis 0 with size 0
        # ---> (손실) 이 없을 경우도 있다.
        QuarterProfit_1_name = html_json.loc[html_json['account_nm']=='당기순이익(손실)', ['thstrm_nm']].values[0]
        QuarterProfit_1_value = html_json.loc[html_json['account_nm']=='당기순이익(손실)', ['thstrm_amount']].values[0]
        QuarterProfit_2_name = html_json.loc[html_json['account_nm']=='당기순이익(손실)', ['frmtrm_nm']].values[0]
        QuarterProfit_2_value = html_json.loc[html_json['account_nm']=='당기순이익(손실)', ['frmtrm_amount']].values[0]
        QuarterProfit_3_name = html_json.loc[html_json['account_nm']=='당기순이익(손실)', ['bfefrmtrm_nm']].values[0]
        QuarterProfit_3_value = html_json.loc[html_json['account_nm']=='당기순이익(손실)', ['bfefrmtrm_amount']].values[0]

        print("############################################################")
        output.write("############################################################\n")
        print("# 당기순이익(손실)")
        output.write("# 당기순이익(손실)\n")
        print("############################################################")
        output.write("############################################################\n")
        print("%s : %s " % (QuarterProfit_1_name, QuarterProfit_1_value))
        output.write("%s : %s\n" % (QuarterProfit_1_name, QuarterProfit_1_value))
        print("%s : %s " % (QuarterProfit_2_name, QuarterProfit_2_value))
        output.write("%s : %s\n" % (QuarterProfit_2_name, QuarterProfit_2_value))
        print("%s : %s " % (QuarterProfit_3_name, QuarterProfit_3_value))
        output.write("%s : %s\n" % (QuarterProfit_3_name, QuarterProfit_3_value))
        print("")
        output.write("\n")
    else :
        print("############################################################")
        output.write("############################################################\n")
        print("# 전기, 전전기 없음")
        output.write("# 전기, 전전기 없음\n")
        print("############################################################")
        output.write("############################################################\n")
        print("")
        output.write("\n")

    print("# Naver WiseRport 분석화면")
    print("https://navercomp.wisereport.co.kr/v2/company/c1030001.aspx?cmp_cd=%s" % Dart_Exp_Code)
    output.write("# Naver WiseRport 분석화면\n")
    output.write("https://navercomp.wisereport.co.kr/v2/company/c1030001.aspx?cmp_cd=%s\n" % Dart_Exp_Code)

def __Dart_Search_Attention(Dart_Corp_Code, Dart_Exp_Code, File_Name, rcpno) :

    if platform == 'darwin' :
        output = open(File_Name, 'a+t')
    else :
        output = open(File_Name, 'a+t', encoding='utf-8')

    BEGIN_Y_DATE = int("2015")
    TODATE_Y_DATE = int(datetime.datetime.today().strftime("%Y"))+1

    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) ' \
                                'AppleWebKit/537.11 (KHTML, like Gecko) ' \
                                'Chrome/23.0.1271.64 Safari/537.11', \
                'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8', \
                'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3', \
                'Accept-Encoding': 'none', \
                'Accept-Language': 'en-US,en;q=0.8', \
                'Connection': 'keep-alive'}

    url = "https://dart.fss.or.kr/dsaf001/main.do?rcpNo=" + rcpno
    print("###############################################1")
    print(url)

    # https://github.com/guineapi9/guineapig/blob/master/재무제표%20크롤링
    req2=requests.get(url)
    html2=req2.text
    soup2=BeautifulSoup(html2, 'html.parser')
    # 재무제표 페이지 찾기
    body=str(soup2.find('head'))
    a=re.search(' 재무제표', body).span()
    print(a)
    # 요약재무정보 근처에서 숫자 정보 찾기(자바스크립트로 되어있지만 정규표현식으로 검색)
    b=re.search(r'viewDoc(.*);',body[a[0]:a[1]+1000]).group()
    print("######## 04")
    # 30번 : XI. 그 밖에 투자자 보호를 위하여 필요한 사항
    print(b)
    # 필요없는 부분 제거
    list=b[8:-2].split(', ')
    list=[i[1:-1] for i in list]
    url_final_1='http://dart.fss.or.kr/report/viewer.do?rcpNo='+list[0]+'&dcmNo='+list[1]+'&eleId='+list[2]+'&offset='+list[3]+'&length='+list[4]+'&dtd=dart3.xsd'
    url_final_2='http://dart.fss.or.kr/report/viewer.do?rcpNo='+list[0]+'&dcmNo='+list[1]+'&eleId='+"30"+'&offset='+list[3]+'&length='+list[4]+'&dtd=dart3.xsd'
    # url = "https://dart.fss.or.kr/report/viewer.do?rcpNo=20200814000814&dcmNo=7442872&eleId=30&offset=678&length=4626&dtd=dart3.xsd"
    # url = "https://dart.fss.or.kr/report/viewer.do?rcpNo=20201116000449&dcmNo=7442872&eleId=30&offset=678&length=4626&dtd=dart3.xsd"
    # url = "https://dart.fss.or.kr/report/viewer.do?rcpNo=20200814000814&dcmNo=7442872&eleId=30&offset=678&length=4626&dtd=dart3.xsd"
    print('재무제표 상태표 url = '+url_final_1)
    output.write('재무제표 상태표 url = ')
    output.write(url_final_1+'\n')
    print('재무제표 투자자 보호 url = '+url_final_2)
    output.write('재무제표 투자자 보호 url = ')
    output.write(url_final_2+'\n')
    # for x in list :
    #     print("########")
    #     print(x)

    output.write("\n")
    output.close()

def __Last_Page_Numb(company_code, dist) :

    if dist == 1 :
        url = "https://finance.naver.com/item/sise_day.nhn?code=" + company_code
    else :
        url = "https://finance.naver.com/research/company_list.nhn?searchType=itemCode&itemCode=" + company_code
    
    print(url)

    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) ' \
        'AppleWebKit/537.11 (KHTML, like Gecko) ' \
        'Chrome/23.0.1271.64 Safari/537.11', \
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8', \
        'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3', \
        'Accept-Encoding': 'none', \
        'Accept-Language': 'en-US,en;q=0.8', \
        'Connection': 'keep-alive'}

    req = Request(url=url, headers=headers)
    html = urlopen(req).read()
    bs_object = bs4.BeautifulSoup(html, "html.parser")
    table = bs_object.find('td', attrs={'class':'pgRR'})
    if table is None :
        print("# Naver Report 없음")
        return 10000

    print(table)

    for x in table.find_all('a') :
        test = x['href']
        active = test.split("=")
        if dist == 1 :
            page_num = active[2]
        else :
            page_num = active[3]

    if dist == 1 :
        print("# Company Page_Number : ", page_num)
    else :
        print("# Report Page_Number : ", page_num)

    return page_num

def __Last_Page_Numb_Market(Market) :
    if Market == "KOSPI" :
        url = "https://finance.naver.com/sise/sise_index_day.nhn?code=KOSPI&page=1"
        # print(url)
    elif Market == "KOSDAQ" :
        url = "https://finance.naver.com/sise/sise_index_day.nhn?code=KOSDAQ&page=1"
        # print(url)

    html = urlopen(url, context=context)
    bs_object = bs4.BeautifulSoup(html, "html.parser")
    main1_pages = bs_object.find_all("a")
    for a_list in main1_pages:
        if a_list is main1_pages[-1] :
            last_page=a_list.attrs['href']
        else :
            continue
        page_num: object=last_page.split('=')
        print("# Market Page_Number : ", page_num[2])
        return page_num[2]

def __Price_Find(company_code, page):
    page = int(page)
    df_list, date_list, closing_list, gap_list, gap_perf  = [], [], [], [], []

    date_list, closing_list = [], []
    # for page_num in range(1, page) :
    for page_num in range(1, 125) : # ---> 1페이지 당 10일, 250일이 년중 평일이라 하면 25페이지가 1년, 125페이지가 5년.
        url= "https://finance.naver.com/item/sise_day.nhn?code=" + company_code + "&page=" + str(page_num)
        print(url)

        req = Request(url=url, headers=headers)

        try :
            html = urlopen(req).read()
            # html = urlopen(url, context=context)
        except HTTPError as err:
            if err.code == 404 :
                print("# Error HTTP : 404")
                return
            elif err.code == 403 :
                print("# Error HTTP : 403")
                return
            elif err.code == 500 :
                print("# Error HTTP : 500")
                return
            elif err.code == 505:
                print("# Error HTTP : 505")
                return
            else:
                raise

        bs_object = bs4.BeautifulSoup(html, "html.parser")
        table = bs_object.find('table', attrs={'class':'type2'})
        table_body = table.find_all('td', attrs={'class':'num'})
        date_body = table.find_all('span', attrs={'class':'tah p10 gray03'})

        for x in date_body :
            # print("x.contents type : %s" % type(x.contents)) # ---> x.contents type : <class 'list'>
            date_list.append(x.contents[0])

        testlist = []
        for tr in table_body :
            td = tr.find_all('span', attrs={'class':'tah p11'})
            # print("for tr type : %s" % type(td))
            for x in td :
                # print("for x in td : %s" % type(x)) # ---> for tr type : <class 'bs4.element.ResultSet'>
                # print("for x.contents in td : %s, %s" % (type(x.contents), x.contents)) # ---> for x.contents in td : <class 'list'>
                # print("for x.contents in td : %s, %s" % (type(x.contents[0]), x.contents[0])) # ---> for x.contents in td : <class 'list'>
                # time.sleep(3)
                # ----> 테이블 중간에 전일비 값이 0이면 table이 엉망이 됨
                # ----> 전일비가 안들어오다가 들어간o
                # print("X Contennts : %s, %s" % (x.contents[0], type(x.contents[0])))
                # print("X Contennts : %s, %s" % (x.contents[0], type(str(x.contents[0]))))
                if str(x.contents[0]) == "0" :
                    continue
                else :
                    testlist.append(x.contents[0])
            
        # print("# TEST LIST")
        # print(testlist)

        for x in range(0, len(testlist)) :
            # print(x, testlist[x])
            if x == 0 :
                closing_list.append(testlist[x])
            else :
                result = x % 5
                if result == 0 :
                    closing_list.append(testlist[x])

    # print("############## date_list ##################")
    # print(date_list)
    # print("############## closing_list ##################")
    # print(closing_list)

    return date_list, closing_list

def __Market_Price(company_code, page):
    df_list, date_list, closing_list = [], [], []
    # print("# Market : %s" % company_code)

    print("# Company : %s" % company_code)
    page = int(page)
    for page_num in range(1, page) :
    # for page_num in range(1, 100) :
        url_main = "https://finance.naver.com/sise/sise_index_day.nhn?code=" + company_code + "&page=" + str(page_num)
        # print("# url : %s" % url_main)
        code_data = pd.read_html(url_main, header=0, skiprows=(6,7,8)) # ---> code data로 출력을 생성하면 NaN이 1개라인이 남음, Type은 <class 'list'>
        df_list.append(pd.read_html(url_main)[0]) # ---> List 로 append, 본문[0]과 페이지 출력[1] 부분 중 본문을 출력

    df=pd.concat(df_list)
    df.dropna(subset=["날짜"], inplace=True) # ---> Naver에서 출력되는 가운데 날짜 컬럼 NaN 라인들 삭제처리
    df.rename(columns={'날짜': 'date'}, inplace = True)
    df.rename(columns={'체결가': 'closing'}, inplace = True)
    df.rename(columns={'전일비': 'diff'}, inplace = True)
    df['new_date'] = pd.to_datetime(df['date']) # ---> dateime 형식으로 new_date 추가
    date_list=df.date.values.tolist()
    closing_list=df.closing.values.tolist()
    for x in range(0, len(closing_list)-1):
        closing_list[x] = int(closing_list[x]) # ---> float 에서 int로 변경

    # date_list.reverse()
    # closing_list.reverse()

    return date_list, closing_list

# def __Make_Chart(date_list, KOSPI_List, KOSDAQ_List, Dart_Exp_Code, Comp_Date_List, Comp_Closing_List, Dart_Comp_Name) :
def __Make_Chart(Dart_Exp_Code, Dart_Comp_Name, Dart_Market_Name) :

    if platform == 'darwin' : # or win32
        DateTime = datetime.datetime.today().strftime("%Y%m%d_%H%M")
        File_Name = "/Users/choiyoungmin/PycharmProjects/2020Y/Anal_Comp/Ananlyst_"+Dart_Exp_Name+"_"+DateTime+".html"
    else :
        DateTime_Win = datetime.datetime.today().strftime("%Y%m%d")
        File_Name = "Analyze"+Dart_Exp_Name+"_"+DateTime_Win+".html"
    
    Comp_Date_List, Comp_Closing_List, KOSPI_DATE_List, KOSPI_List, KOSDAQ_DATE_List, KOSDAQ_List = [], [], [], [], [], []
    page = __Last_Page_Numb(Dart_Exp_Code, 1)
    KOSPI_Page = __Last_Page_Numb_Market("KOSPI")
    KOSDAQ_Page = __Last_Page_Numb_Market("KOSDAQ")

    Comp_Date_List, Comp_Closing_List = __Price_Find(Dart_Exp_Code, page)
    print(Comp_Date_List)
    print(Comp_Closing_List)
    KOSPI_Date, KOSPI_List = __Market_Price("KOSPI", KOSPI_Page)
    KOSDAQ_Date, KOSDAQ_List = __Market_Price("KOSDAQ", KOSDAQ_Page)

    print("# KOSPI_DATE : %s, length : %s" % (KOSPI_Date[0], len(KOSPI_Date)))
    print("# KOSPI_List : %s, length : %s" % (KOSPI_List[0], len(KOSPI_List)))
    print("# KOSDAQ_DATE : %s, length : %s" % (KOSDAQ_Date[0], len(KOSDAQ_Date)))
    print("# KOSDAQ_List : %s, length : %s" % (KOSDAQ_List[0], len(KOSDAQ_List)))
    print("# Comp DATE : %s, length : %s" % (Comp_Date_List[0], len(Comp_Date_List)))
    print("# Comp Closing : %s, length : %s" % (Comp_Closing_List[0], len(Comp_Date_List)))
    print("")

    if Dart_Market_Name == '코스피' :
        date_list = KOSPI_Date
        if len(Comp_Date_List) < len(date_list) : # ---> 코스피 기간이 더 길어서 코스피 삭제
            Date_Index = date_list.index(Comp_Date_List[0])
            print(Date_Index)
            del date_list[0:Date_Index]
            del KOSPI_List[0:Date_Index]
        else : # ---> 종목 기간이 더 길어서 종목 기간 삭제 ---> 시장보다 길다???
            Date_Index = KOSPI_List.index(date_list[0])
            print(Date_Index)
            del Comp_Date_List[0:Date_Index]
            del Comp_Closing_List[0:Date_Index]
    elif Dart_Market_Name == '코스닥' :
        date_list = KOSDAQ_Date
        if len(Comp_Date_List) < len(date_list) : # ---> 코스닥 기간이 더 길어서 코스피 삭제
            Date_Index = date_list.index(Comp_Date_List[0])
            print(Date_Index)
            del date_list[0:Date_Index]
            del KOSDAQ_List[0:Date_Index]
        else : # ---> 종목 기간이 더 길어서 종목 기간 삭제 ---> 시장보다 길다???
            Date_Index = KOSDAQ_List.index(date_list[0])
            print(Date_Index)
            del Comp_Date_List[0:Date_Index]
            del Comp_Closing_List[0:Date_Index]
    else :
        date_list = KOSPI_Date

    print("# KOSPI_DATE : %s, length ; %s" % (KOSPI_Date[0], len(KOSPI_Date)))
    print("# KOSPI_List : %s, length ; %s" % (KOSPI_List[0], len(KOSPI_List)))
    print("# KOSDAQ_DATE : %s, length ; %s" % (KOSDAQ_Date[0], len(KOSDAQ_Date)))
    print("# KOSDAQ_List : %s, length ; %s" % (KOSDAQ_List[0], len(KOSDAQ_List)))
    print("# Comp DATE : %s, length ; %s" % (Comp_Date_List[0], len(Comp_Date_List)))
    print("# Comp Closing : %s, length ; %s" % (Comp_Closing_List[0], len(Comp_Date_List)))
    print("")

# KOSPI_DATE : 1990.01.06
# KOSPI_List : 912.86
# KOSDAQ_DATE : 1996.07.08
# KOSDAQ_List : 1048.2
# Comp DATE : 2010.01.08

    print("# date list : %s" % date_list[0])

    input_1 = open('Echart_Test_1.html', 'r') # Echrt의 머릿글
    input_2 = open('Echart_Test_2.html', 'r') # Echart의 끝글

    # if Comp_Date_List in date_list :
    #     Date_Index = date_list.index(Comp_Date_List[0])
    #     # ---> ValueError: '2010.01.08' is not in list
    #     # KOSPI/KODAQ date list : 2014.05.21
    #     # print("# Index : %s" % Date_Index)
    #     # print("# Value : %s" % date_list[Date_Index])
    #     # print("Comp Closing list : %s" % len(Comp_Closing_List))
    #     for x in range(0, int(Date_Index)) :
    #         # print("# %s번째 " % x)
    #         Comp_Closing_List.insert(0, "0") # 0번째에 넣은 리스트에 또 앞자리에 넣게된다. 
    # else :
    #     print("date_list index : %s" % date_list.index("2014.05.21"))
    #     Date_Index = Comp_Date_List.index(date_list.index("2014.05.21"))
    #     print("Date Index : %s" % Date_Index)
    #     del Comp_Closing_List[0:Date_Index]

    closing_list_1 = KOSPI_List
    closing_list_2 = KOSDAQ_List
    closing_list_3 = Comp_Closing_List

    # print("date list : %s" % len(date_list))
    # print("KOSPI list : %s" % len(KOSPI_List))
    # print("KOSDAQ list : %s" % len(KOSDAQ_List))
    # print("Comp Date list : %s" % len(Comp_Date_List))
    # print("Comp Closing list : %s" % len(Comp_Closing_List))

    if platform == 'darwin' :
        output = open(File_Name, 'a+t')
    else :
        output = open(File_Name, 'a+t', encoding='utf-8')

    print("# MAKE HTML Start")
    for line in input_1.readlines():
        output.write(line)

    output.write("          xAxis: {\n")
    output.write("          data: \n")
    output.write("[")
    for item in date_list:
        output.write("'%s'," % item)
    output.write("]")
    output.write("\n")
    output.write("          },\n")
    output.write("          yAxis: {},\n")
    output.write("          series: [\n")
    if Dart_Market_Name == '코스피' :
        output.write("          {\n")
        output.write("              name: '%s',\n" % "KOSPI")
        output.write("              type: '%s',\n" % "line")
        output.write("              data: ")
        output.write("[")
        for x in closing_list_1 :
            output.write("'%s'," % x)
        output.write("]")
        output.write("              \n")
        output.write("          },\n")
    elif Dart_Market_Name == '코스닥' :
        output.write("          {\n")
        output.write("              name: '%s',\n" % "KOSDAQ")
        output.write("              type: '%s',\n" % "line")
        output.write("              data: ")
        output.write("[")
        for x in closing_list_2 :
            output.write("'%s'," % x)
        output.write("]")
        output.write("              \n")
        output.write("          },\n")
    else :
        output.write("          {\n")
        output.write("              name: '%s',\n" % "KOSPI")
        output.write("              type: '%s',\n" % "line")
        output.write("              data: ")
        output.write("[")
        for x in closing_list_1 :
            output.write("'%s'," % x)
        output.write("]")
        output.write("              \n")
        output.write("          },\n")
    output.write("          {\n")
    output.write("              name: '%s',\n" % Dart_Comp_Name)
    output.write("              type: '%s',\n" % "line")
    output.write("              data: ")
    output.write("[")
    for x in closing_list_3 :
        output.write("'%s'," % x)
    output.write("]")
    output.write("              \n")
    output.write("          },\n")

    for line in input_2.readlines():
        output.write(line)

    print("# MAKE HTML Done")

    output.close()

# def __New_Make_Chart (date_list, closing_list_A, closing_list_B) :
def __New_Make_Chart (Dart_Exp_Code, Dart_Comp_Name, Market, Market_Date, Market_List) :

    if platform == 'darwin' : # or win32
        DateTime = datetime.datetime.today().strftime("%Y%m%d_%H%M")
        File_Name = "/Users/choiyoungmin/PycharmProjects/2020Y/Anal_Comp/Ananlyst_"+Dart_Comp_Name+"_"+DateTime+".html"
        input_1 = open('/Users/choiyoungmin/PycharmProjects/2020Y/Anal_Comp/New_Echart_Test_1.html', 'r', encoding='utf-8') # Echrt의 머릿글
    else :
        DateTime_Win = datetime.datetime.today().strftime("%Y%m%d")
        File_Name = "Analyze"+Dart_Comp_Name+"_"+DateTime_Win+".html"
        input_1 = open('New_Echart_Test_1.html', 'r', encoding='utf-8') # Echrt의 머릿글
    # print(File_Name)

    # 아래는 Make_Chart를 위한 조건들
    Comp_Date_List, KOSPI_DATE_List, KOSDAQ_DATE_List = [], [], []
    Comp_Closing_List, KOSPI_List, KOSDAQ_List = [], [], []

    Exp_page = __Last_Page_Numb(Dart_Exp_Code, 1)
    Comp_Date_List, Comp_Closing_List = __Price_Find(Dart_Exp_Code, Exp_page)
    date_list = Comp_Date_List

    print("Market : %s" % Market)

    # if Market == "코스피" :
    #     KOSPI_Page = __Last_Page_Numb_Market("KOSPI")
    #     KOSPI_Date, KOSPI_List = __Market_Price("KOSPI", KOSPI_Page)
    #     closing_list_A = KOSPI_List
    #     closing_list_B = Comp_Closing_List
    # elif Market == "코스닥" :
    #     KOSDAQ_Page = __Last_Page_Numb_Market("KOSDAQ")
    #     KOSDAQ_Date, KOSDAQ_List = __Market_Price("KOSDAQ", KOSDAQ_Page)
    #     closing_list_A = KOSDAQ_List
    #     closing_list_B = Comp_Closing_List

    closing_list_A = Market_List
    closing_list_B = Comp_Closing_List

    # n = 200
    n = len(closing_list_B)
    print("date list 0 : %s" % date_list[0])
    print("date leng : %s" % len(date_list))
    print("closing list A-0 : %s " % closing_list_A[0])
    print("closing leng A : %s " % len(closing_list_A))
    print("closong list B-0 : %s " % closing_list_B[0])
    print("closing leng B : %s " % len(closing_list_B))
    del date_list[n:]
    del closing_list_A[n:]
    del closing_list_B[n:]
    print("# After DEL")
    print("date list 0 : %s" % date_list[0])
    print("closing list A-0 : %s " % closing_list_A[0])
    print("closing leng A : %s " % len(closing_list_A))
    print("closong list B-0 : %s " % closing_list_B[0])
    print("closing leng B : %s " % len(closing_list_B))

    if platform == 'darwin' :
        output = open(File_Name, 'a+t')
    else :
        output = open(File_Name, 'a+t', encoding='utf-8')

    # ls = [type(item) for item in closing_list_A]
    # print(ls) # ---> int
    # ls = [type(item) for item in closing_list_B]
    # print(ls) # ---> <class 'bs4.element.NavigableString'>
    # 천 단위 , 자리수가 있으면 string이고 없으면 int 로 list 생성되는듯..

    for idx, val in enumerate(closing_list_A) :
        if type(closing_list_A[idx]) == int or type(closing_list_A[idx]) == float :
            continue
        else :
            val = val.replace(",","")
            closing_list_A[idx] = val

    for idx, val in enumerate(closing_list_B) :
        if type(closing_list_B[idx]) == int or type(closing_list_B[idx]) == float :
            continue
        else :
            val = val.replace(",","")
            closing_list_B[idx] = val

    # print(date_list.reverse()) # ---> None
    # print(reversed(date_list)) # ---> <list_reverseiterator object at 0x12235bbb0>
    # date_list.reverse() # ---> None
    # print(reversed(date_list)) # ---> <list_reverseiterator object at 0x1190a7d00>
    # print(date_list[::-1]) # ---> OK

    # max_value_A = int(max(closing_list_A)) * 2
    # max_value_B = int(max(closing_list_B)) * 2
    numrow_A = [float(x) for x in closing_list_A]
    max_value_A = int(max(numrow_A)) * 1.5
    numrow_B = [float(x) for x in closing_list_B]
    max_value_B = int(max(numrow_B)) * 1.5 # ---> ValueError: could not convert string to float: '11,550'

    print("# MAKE HTML Start")
    for line in input_1.readlines():
        output.write(line)
    output.write("        data: ['%s', '%s']\n" % (Market, Dart_Comp_Name))
    output.write("    },\n")
    output.write("    xAxis: [\n")
    output.write("        {\n")
    output.write("            type: 'category',\n")
    output.write("            axisTick: {\n")
    output.write("                alignWithLabel: true\n")
    output.write("            },\n")
    output.write("            data: [")
    for item in date_list[::-1] :
        output.write("'%s'," % item)
    output.write("        ]\n")
    output.write("        }\n")
    output.write("    ],\n")
    output.write("    yAxis: [\n")
    output.write("        {\n")
    output.write("            type: 'value',\n")
    output.write("            name: '%s',\n" % Market)
    output.write("            min: 0,\n")
    output.write("            max: %s,\n" % str(max_value_A))
    output.write("            position: 'right',\n")
    output.write("            axisLine: {\n")
    output.write("                lineStyle: {\n")
    output.write("                    color: colors[0]\n")
    output.write("                }\n")
    output.write("            },\n")
    output.write("            axisLabel: {\n")
    output.write("                formatter: '{value} 원'\n")
    output.write("            }\n")
    output.write("        },\n")
    output.write("        {\n")
    output.write("            type: 'value',\n")
    output.write("            name: '%s',\n" % Dart_Comp_Name)
    output.write("            min: 0,\n")
    output.write("            max: %s,\n" % str(max_value_B))
    output.write("            position: 'left',\n")
    output.write("            axisLine: {\n")
    output.write("                lineStyle: {\n")
    output.write("                    color: colors[1]\n")
    output.write("                }\n")
    output.write("            },\n")
    output.write("            axisLabel: {\n")
    output.write("                formatter: '{value} 원'\n")
    output.write("            }\n")
    output.write("        }\n")
    output.write("    ],\n")
    output.write("    series: [\n")
    output.write("        {\n")
    output.write("            name: '%s',\n" % Market)
    output.write("            type: 'line',\n")
    output.write("              data: [\n")
    for x in closing_list_A[::-1] :
        output.write("'%s'," % x)
    output.write("        ]\n")
    output.write("        },\n")
    output.write("        {\n")
    output.write("            name: '%s',\n" % Dart_Comp_Name)
    output.write("            type: 'line',\n")
    output.write("            yAxisIndex: 1,\n")
    output.write("              data: [\n")
    for x in closing_list_B[::-1]  :
        output.write("'%s'," % x)
    output.write("    ]\n")
    output.write("        }\n")
    output.write("    ]\n")
    output.write("};\n")
    output.write("        // use configuration item and data specified to show chart\n")
    output.write("        myChart.setOption(option);\n")
    output.write("    </script>\n")
    output.write("</body>\n")

    # for line in input_2.readlines():
        # output.write(line)

    print("# MAKE HTML Done")

    output.close()

def __Naver_Research(Dart_Comp_Name, Dart_Exp_Code, File_Name) :

    # File_Name_1 = "/Users/choiyoungmin/PycharmProjects/2020Y/Anal_Comp/Naver_Reports_Subject.txt"
    # File_Name_2 = "/Users/choiyoungmin/PycharmProjects/2020Y/Anal_Comp/Naver_Reports_Files.txt"
    # Date_Check = datetime.datetime.today().strftime("%y.%m.%d")

    # f = open(File_Name_1, "r")
    # searchlines = f.readlines()
    # f.close()

    # ############################################################  
    # # 기존 레포트 파일에서 날짜 검색
    # ############################################################  
    # for i, line in enumerate(searchlines):
    #     if i == 0 :
    #         x = line.split()
    #         # print(x) # ---> ['1', '대양전기공업', '수소차로', '상륙하다', '교보증권', 'NaN', '21.01.20', '3968.0']
    #         Date_Check = x[-2] # ---> 뒤에서 조회수, 날짜...앞에서 하면 제목의 띄워쓰기때문에...
    #         StockComp_Check = x[-4] # ---> 뒤에서 조회수, 날짜...앞에서 하면 제목의 띄워쓰기때문에...
    #         Subject_Check = x[-5] # ---> 뒤에서 조회수, 날짜...앞에서 하면 제목의 띄워쓰기때문에...
    #         break # ---> 라인 시작이 오늘이 아니면 체크 끝

    # print("# Check Date : %s" % Date_Check)
    # print("# Check StockComp : %s" % StockComp_Check)
    # print("# Check Subject : %s" % Subject_Check)
    
    # # for i, line in enumerate(searchlines):
    # #     if Date_Check not in line and i == 0 :
    # #     # if Date_Check in line and i == 0 :
    # #         print("Date_Check")
    # #         print(Date_Check)
    # #         print("# 오늘....")
    # #         print("# %s 라인 : %s" % (i, line))
    # #         break # ---> 라인 시작이 오늘이 아니면 체크 끝
    # #     # 1    대양전기공업                       수소차로 상륙하다     교보증권 NaN  21.01.20   3879.0
    # # return 0

    # File_Name_1 = "/Users/choiyoungmin/PycharmProjects/2020Y/Anal_Comp/Naver_Reports_Subject_New.txt"
    # File_Name_2 = "/Users/choiyoungmin/PycharmProjects/2020Y/Anal_Comp/Naver_Reports_Files_New.txt"

    # output_1 = open(File_Name_1, 'a+t')
    # output_2 = open(File_Name_2, 'a+t')

    # url = "https://finance.naver.com/research/company_list.nhn"
    # url = "https://finance.naver.com/research/company_list.nhn?searchType=itemCode&itemCode=" + Dart_Exp_Code

    # html = urlopen(url, context=context)
    # bs_object = bs4.BeautifulSoup(html, "html.parser")
    # main_pages = bs_object.find_all("a")

    page_max = __Last_Page_Numb(Dart_Exp_Code, 2)
    if page_max == 10000 :
        return None
    print("# page_max : %s" % page_max)

    Report_Total = pd.DataFrame()
    File_List = []
    for page_num in range(1, int(page_max)+1) :
    # for page_num in range(1, 6) :

        url = "https://finance.naver.com/research/company_list.nhn?searchType=itemCode&itemCode=" + Dart_Exp_Code + "&page=" + str(page_num)
        # print(url)

        headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) ' \
                                'AppleWebKit/537.11 (KHTML, like Gecko) ' \
                                'Chrome/23.0.1271.64 Safari/537.11', \
                'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8', \
                'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3', \
                'Accept-Encoding': 'none', \
                'Accept-Language': 'en-US,en;q=0.8', \
                'Connection': 'keep-alive'}
        req = Request(url=url, headers=headers)

        try :
            html = urlopen(req).read()
            # html = urlopen(url, context=context)
        except HTTPError as err:
            if err.code == 404 :
                print("# Error HTTP : 404")
                return
            elif err.code == 403 :
                print("# Error HTTP : 403")
                return
            elif err.code == 500 :
                print("# Error HTTP : 500")
                return
            elif err.code == 505:
                print("# Error HTTP : 505")
                return
            else:
                raise

        bs_object = bs4.BeautifulSoup(html, "html.parser")

        table = bs_object.find('div', {'class':'box_type_m'})
        file_table = table.find_all('td', {'class':'file'}) # ---> find로하면 첫번째만 검색

        # ---> TypeError: 'NoneType' object is not subscriptable
        # File_SR = pd.Series() # ---> DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
        # print(type(file_table)) # ---> find <class 'bs4.element.Tag'>, find_all <class 'bs4.element.ResultSet'>
        # print(file_table)
        for a_list in file_table : 
            # print("# a_list : %s, %s" % (a_list, type(a_list))) # ---> # a_list : <td class="file"></td>, <class 'bs4.element.Tag'>
            try:
                file_link = a_list.find("a")["href"]
                File_List.append(file_link)
                # print("# file_list : %s" % file_link) # --->             # TypeError: 'NoneType' object is not subscriptable
            except TypeError:
                File_List.append("N/A")
                pass

        table_data = pd.read_html(str(table), header=0) # ---> pd.read_html은 List 형태로 보내고 그 안에 DataFrame
        Report_Data = table_data[0] # ---> [1] 일 경우는 하단의 페이지 번호가 출력
        # ---> Columns: [종목명, 제목, 증권사, 첨부, 작성일, 조회수]
        Report_Data.dropna(subset=['종목명'], inplace=True)
        # Report_Data["레포트"] = File_List
        # print(type(Report_Data)) # ---> <class 'pandas.core.frame.DataFrame'>

        # print(Report_Data.loc[Report_Data['작성일']=="21.01.22", ['작성일']].tail(1))
        # print(Report_Data.loc[Report_Data['작성일']=="21.01.22", ['작성일']].tail(1).values[0])
        # ---> ['21.01.22'] , 값이 없으면 out of index에러
        # ---> IndexError: index 0 is out of bounds for axis 0 with size 0

        # Report_Total.append(Report_Data, ignore_index = True)
        Report_Total = pd.concat([Report_Total, Report_Data], ignore_index=True)

        # output_1.write(Report_Data.to_string(header=True, index=True))
        # output_1.write("\n")

        # for idx, file_write in enumerate(File_List) :
        #     output_2.write("%s\t%s\n" % (idx, file_write))

    # Report_Total.reset_index()
    # print("# Report_Data")
    # print(Report_Total)
    # print("# File List")
    # print(len(File_List))
    # print(File_List)

    for index, row in Report_Total.iterrows() :
        # print("# index : %s : " % index)
        print("# %s : %s - %s - %s" % (row['종목명'], row['제목'], row['작성일'], row['증권사']))
        print(File_List[index])
        # print(File_List[index-1])

    # File_Name_1 = "/Users/choiyoungmin/PycharmProjects/2020Y/Anal_Comp/Naver_Reports_Subject.txt"
    # File_Name_2 = "/Users/choiyoungmin/PycharmProjects/2020Y/Anal_Comp/Naver_Reports_Files.txt"

    ############################################################
    # 아래는 레포트 파일에서 기업검색
    ############################################################

    # f = open(File_Name_1, "r")
    # searchlines = f.readlines()
    # f.close()

    # print(Dart_Comp_Name)

    # Subject_line = -1 # ---> list든 file read든 0부터 시작이니 -1로 미리 선언
    # Subject_line_List = [] # ---> 레포트가 다수일 경우 리스트를 해야한다.

    # for i, line in enumerate(searchlines):
    #     if Dart_Comp_Name in line: # 라인에 회사이름과 라인넘버가 같으면 있는 것으로 간주
    #         print("# %s 라인 : %s" % (i, line))
    #         # Subject_line = i
    #         Subject_line_List.append(i)

    # f = open(File_Name_2, "r")
    # searchlines = f.readlines()
    # f.close()
    # for i, line in enumerate(searchlines):
    #     # if Subject_line == i : # dnl
    #     if i in Subject_line_List :
    #         print("# %s 라인 : %s" % (i, line))

def __Google_Upload(File_Name):

    creds = None

    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('drive', 'v3', credentials=creds)

    _Name = datetime.datetime.today().strftime("_%Y%m%d")

    Upload_File_List = [File_Name]
    print(Upload_File_List)
    
    for file_name in Upload_File_List :
        print("Upload File : %s" % file_name)
        file_metadata = {'name': file_name, 'parents' : ['18BXSAPuOYhd_P0DGIdwLbLCW7XNPcAZh']}
        media = MediaFileUpload(file_name, resumable=True) # mimetype은 자동으로 정의해주므로 삭제
        file = service.files().create(body=file_metadata, media_body=media, fields='id').execute()
        print("# File ID: %s" % file.get('id'))

def __NPS_Check() :

    print("# 국민연금 Check")
    NPS_DF = pd.read_csv('Anal_Comp/NPS_Check.xlsx', error_bad_lines=False)
    # ---> pandas.errors.ParserError: Error tokenizing data. C error: Expected 4 fields in line 158, saw 6
    #      ---> error_bad_lines=False)
    print(NPS_DF)


def __MENU() : 
    print("")
    print("##############################")
    print("# Choice Company Order : 0 ~ 9")
    print("")
    input_V = input()
    return input_V

if __name__ == '__main__' :
    print("############################################################")
    print("# Program Start")
    print("############################################################")

    BEGIN_DATE = "20100101"
    ROE_Min = 30
    # PER_Min, PER_Max = 1, 10
    PER_Min, PER_Max = 1, 20
    TOP_No = 30
    print("############################################################")
    print("# 10 페이지 조회, 1페이지 당 50개, 총 500개")
    print("# ROE_Min = %s" % ROE_Min)
    print("# PER_Min, PER_Max = %s, %s" % (PER_Min, PER_Max))
    print("############################################################")

    TEST_Go = 0
    # TEST_Go = 1 # 밑에 것들만 TEST
    
    if TEST_Go != 1 :
        Result_Comp = pd.DataFrame()
        for Market in range(1,3): # KOSPI, KOSDAQ 을 각각 네이버에서 호출하여 Result_Comp로 합친다
        # for x in range(1,2):
            Get_Comp = __Get_Result(Market, ROE_Min)
            Result_Comp = Result_Comp.append(Get_Comp)
            # print(Result_Comp)
            ################## 여기서 무언가 없어진다.......................

        # 이전에서 안된 ROE, PER 조건을 다시 주고 Sorting 도 다시 시행
        # Sorting을 다시해서 KOSPI&KOSDAQ 섞어서 Sorting됨 ---> KOSPI가 찾기 어려워지는...ㅠ
        Result_Comp = Result_Comp.loc[(Result_Comp['ROE'] >= ROE_Min)]
        Result_Comp = Result_Comp.loc[(Result_Comp['PER'] > PER_Min)&(Result_Comp['PER'] < PER_Max)]
        Result_Comp = Result_Comp.sort_values(by=['ROE'], ascending=False)

        # print("# Result_Comp")
        # print(Result_Comp)

    Corp_DF =  __get_XML()
    # print(Corp_DF[Corp_DF['corp_name']=="삼성전자"])
    # print(Corp_DF[Corp_DF['corp_name']=="삼성전자"].values)
    # print(Corp_DF[Corp_DF['corp_name']=="삼성전자"].values[0])
    # print(Corp_DF.loc[Corp_DF.corp_name=="한솔케미칼",'stock_code'].values[0])
    # print(Corp_DF.loc[Corp_DF.corp_name=="교촌에프앤비",'stock_code'].values[0])

    Result_Comp_New = pd.DataFrame()
    Result_PBR_List, Recently_PER_List, Recently_EPS_List = [], [], []
    Expected_PER_List, Expected_EPS_List, Recently_PBR_List = [], [], []
    Field_PER_List, Expected_Value_List = [], []
    if TEST_Go != 1 :
        for index, row in Result_Comp.iterrows():
            # row_expcode = Corp_DF.loc[Corp_DF.corp_name==row['회사명'],'stock_code'].values
            # row_expcode = Corp_DF.loc[Corp_DF.corp_name==row['회사명'],'stock_code'].values[0]
            # ---> IndexError: index 0 is out of bounds for axis 0 with size 0
            # ---> 화승인더
            # ---> Empty DataFrame
            row_test = Corp_DF.loc[Corp_DF.corp_name==row['회사명']]
            if row_test.empty :
            # if row_expcode.empty : # ---> AttributeError: 'str' object has no attribute 'empty'
            # if row_expcode == '' :
                # print("DataFrame is empty! : %s " % row['회사명'])
                continue
            else :
                # print("회사명 : %s , %s, %s" % (row['회사명'], Corp_DF.loc[Corp_DF.corp_name==row['회사명'],'stock_code'].values[0], Corp_DF.loc[Corp_DF.corp_name==row['회사명'],'corp_code'].values[0]))
                # print(Corp_DF.loc[Corp_DF.corp_name==row['회사명'],'stock_code'].values[0])
                # print(Corp_DF.loc[Corp_DF.corp_name==row['회사명'],'corp_code'].values[0])
                row_expcode = Corp_DF.loc[Corp_DF.corp_name==row['회사명'],'stock_code'].values[0]
                row_corpcode = Corp_DF.loc[Corp_DF.corp_name==row['회사명'],'corp_code'].values[0]
                Result_Comp.loc[index,'expcode'] = row_expcode
                Result_Comp.loc[index,'corpcode'] = row_corpcode
                # print(row['회사명'], row['ROE'], row['PER'], row['외국인비율'], row['Market'], row_expcode, row_corpcode)
                # 이상하게 여기서는 정상이지만 여기 for문과 if문을 빠져나가면 XML 하고 맞춘 코드가 엉망이 된다...ㅠㅠ
                # 그래서 DataFrame 다시 생성...
                new_row = {'회사명':row['회사명'], 'ROE':row['ROE'], 'PER':row['PER'], '현재가':row['현재가'], '외국인비율':row['외국인비율'], 'Market':row['Market'] , 'expcode':row_expcode, 'corpcode':row_corpcode}
                Result_Comp_New = Result_Comp_New.append(new_row, ignore_index=True)

        for index, row in Result_Comp_New.iterrows():
            Result_PBR = __Naver_Inst_Anal(row['expcode'])
            Result_Num = __Naver_Per_Info(row['expcode'])
            # print(row['expcode'])
            # print(Result_Num)
            # Result_Num : List # PER, EPS --> 최근4분기, cns_Per, cns_EPS ---> 추정, _PBR, BPS, 배당수익율, 동일업종PER
            Result_PBR_List.append(Result_PBR)
            Recently_PER = Result_Num[0]
            Recently_PER_List.append(Recently_PER)
            Recently_EPS = Result_Num[1]
            Recently_EPS_List.append(Recently_EPS)
            Expected_PER = Result_Num[2]
            Expected_PER_List.append(Expected_PER)
            Expected_EPS = Result_Num[3]
            Expected_EPS_List.append(Expected_EPS)
            Recently_PBR = Result_Num[4]
            Recently_PBR_List.append(Recently_PBR)
            Field_PER = Result_Num[7]
            Field_PER_List.append(Field_PER)

            Recently_EPS_1 = Recently_EPS.replace(",", "")
            Field_PER_1 = Field_PER.replace(",", "")
            Expected_Value = float(Recently_EPS_1) * float(Field_PER_1)
            Expected_Value_List.append(f'{int(Expected_Value):,}')

        # print("# Result_PBR_LIST")
        # print(len(Result_PBR_List))
        # print(Result_PBR_List)
        # print("# PBR")
        # print(Result_Comp_New)
        Result_Comp_New['PBR'] = Result_PBR_List
        for i in Recently_PER_List :
            float(i)
        Result_Comp_New['최근PER'] = Recently_PER_List
        for i in Recently_PBR_List :
            float(i) 
        Result_Comp_New['최근PBR'] = Recently_PBR_List
        Result_Comp_New['최근EPS'] = Recently_EPS_List
        Result_Comp_New['예상PER'] = Expected_PER_List
        Result_Comp_New['예상EPS'] = Expected_EPS_List
        Result_Comp_New['업종PER'] = Field_PER_List
        Result_Comp_New['예상시가'] = Expected_Value_List

        Result_Comp_New['최근PBR_1'] = Result_Comp_New['최근PBR'].astype(float)
        Result_Comp_New['최근PER_1'] = Result_Comp_New['최근PER'].astype(float)
        Result_Comp_New['PBR*PER=22.5'] = Result_Comp_New['최근PBR_1'] * Result_Comp_New['최근PER_1']
        Result_Comp_New['PBR*PER=22.5'] = Result_Comp_New['PBR*PER=22.5'].astype(int)
        Result_Comp_New.drop(columns=['최근PBR_1','최근PER_1'])
        Result_Comp_New['예상시가_th'] = Result_Comp_New['예상시가'].str.replace(',','')
        Result_Comp_New['현재가_th'] = Result_Comp_New['현재가'].str.replace(',','')
        Result_Comp_New['예상시가_th'] = Result_Comp_New['예상시가_th'].astype(int)
        Result_Comp_New['현재가_th'] = Result_Comp_New['현재가_th'].astype(int)
        Result_Comp_New['GAP'] = round(Result_Comp_New['예상시가_th'] / Result_Comp_New['현재가_th'] * 100, 1)
        Result_Comp_New['ROE_RANK'] = Result_Comp_New['ROE'].rank()


        # Result_Comp_New['PBR*PER=22.5'] = Result_Comp_New['최근PBR'] * Result_Comp_New['최근PER']
        # ---> TypeError: can't multiply sequence by non-int of type 'str'
        # Result_Comp_New['PBR*PER=22.5'] = int(Result_Comp_New['최근PBR']) * int(Result_Comp_New['최근PER'])
        # ---> TypeError: cannot convert the series to <class 'int'>

        # TypeError: can't multiply sequence by non-int of type 'str'
        # df['DataFrame Column'] = df['DataFrame Column'].astype(int)
        # Result_Comp_New['최근EPS'] = Result_Comp_New['최근EPS'].astype(float)
        # Result_Comp_New['현재가_1'] = Result_Comp_New['최근EPS'].astype(float)
        # Result_Comp_New['예상현재가'] = Result_Comp_New['최근EPS_1'] * Result_Comp_New['현재가_1']
        # Result_Comp_New['현재가차이'] = (Result_Comp_New['예상현재가'] / Result_Comp_New['현재가']) * 100
        # Result_Comp_New.drop(['최근EPS_1'], ['현재가_1'], inplace=True)

    if TEST_Go != 1 :
        print("############################################################")
        print("# Market Price List Getting...")
        KOSPI_Page = __Last_Page_Numb_Market("KOSPI") # ---> 최근 1년이면 굳이 모두 할 필요없다.
        KOSPI_Date, KOSPI_List = __Market_Price("KOSPI", KOSPI_Page)
        KOSDAQ_Page = __Last_Page_Numb_Market("KOSDAQ")
        KOSDAQ_Date, KOSDAQ_List = __Market_Price("KOSDAQ", KOSDAQ_Page) # ---> 최근 1년이면 굳이 모두 할 필요없다.
        print("# Market Price List Done...")

        print("############################################################")
        print("# TOP 30")
        print("# ROE_Min = %s" % ROE_Min)
        print("# PER_Min, PER_Max = %s, %s" % (PER_Min, PER_Max))
        print("############################################################")
        # print(Result_Comp.head(TOP_No))
        # print(Result_Comp_New[['회사명', 'ROE', 'PER', 'PBR', '최근PER', '최근EPS', '업종PER', '현재가', '예상시가', '외국인비율', 'Market', 'expcode', 'corpcode']].head(TOP_No))
        # print(tabulate(df, headers='keys', tablefmt='psql'))
        print(tabulate(Result_Comp_New[['회사명', 'ROE', 'PER', 'PBR', '최근PER', 'PBR*PER=22.5', '최근EPS', '업종PER', '현재가', '예상시가', 'GAP', '외국인비율', 'Market', 'expcode', 'corpcode']].head(TOP_No),headers='keys',tablefmt='psql'))
        print("############################################################")
        print("")

        DateTime = datetime.datetime.today().strftime("%Y%m%d")
        Write_TEXT = Result_Comp_New[['회사명', 'ROE', 'PER', 'PBR', '최근PBR', '최근PER', 'PBR*PER=22.5', '최근EPS', '업종PER', '현재가', '예상시가', 'GAP', '외국인비율', 'Market', 'expcode', 'corpcode']].head(TOP_No)
        # print("# TESTTESTTESTTEST TEST")
        # '<head><meta charset="utf-8"></head>' + Write_TEXT.to_html()
        # Write_TEXT.encoding = 'EUC-KR'

        if platform == 'darwin' : # or win32
            DateTime = datetime.datetime.today().strftime("%Y%m%d_%H%M")
            File_Name_tabulate = "/Users/choiyoungmin/PycharmProjects/2020Y/Anal_Comp/Ananlyst_"+DateTime+"_tabulate.html"
        else :
            DateTime_Win = datetime.datetime.today().strftime("%Y%m%d")
            File_Name_tabulate = "Analyze"+DateTime_Win+"_tabulate.html"

        with io.open(File_Name_tabulate, 'w', encoding='utf-8') as f:
            # f.write(tabulate(Write_TEXT, tablefmt="html"))
            # f.write('<head><meta charset="utf-8"></head>' + Write_TEXT.to_html())
            f.write('<head><meta charset="utf-8"></head>')
            f.write('<style>')

            # f.write('table {')
            # f.write('width: 70%;')
            # f.write('border: 1px solid #444444;')
            # f.write('border-collapse: collapse;')
            # f.write('}')
            # f.write('th, td {')
            # f.write('border: 1px solid #444444;')
            # f.write('padding: 10px; text-align: center')
            # f.write('}')

            f.write('table {border: 1px solid #444444;border-collapse: collapse}')
            f.write('th, td {text-align: center}')

            f.write('</style>')

            f.write(Write_TEXT.to_html())
            # f.write(tabulate(Write_TEXT, tablefmt="html")) # table의 헤더가 안들어간다..ㅠ
            # f.write(tabulate(Write_TEXT, tablefmt="text"))
        f.close()

    # Dart_Corp_Code = str(Result_Comp_New[['corpcode']].head(1).replace(" ", ""))
    # Dart_Exp_Code = str(Result_Comp_New[['expcode']].head(1).replace(" ", ""))

    if TEST_Go != 1 :
        loop = 1
        while loop == 1:
            print(tabulate(Result_Comp_New[['회사명', 'ROE', 'PER', 'PBR', '최근PBR', '최근PER', 'PBR*PER=22.5', '최근EPS', '업종PER', '현재가', '예상시가', 'GAP', '외국인비율', 'Market', 'expcode', 'corpcode']].head(TOP_No),headers='keys',tablefmt='psql'))
            Choice_Comp = __MENU()
            print(type(Choice_Comp))
            print("# 다시 시작...")
            if Choice_Comp.isdigit() :
                Comp_Name = Result_Comp_New['회사명'].iloc[int(Choice_Comp)]
                if platform == 'darwin' : # or win32
                    DateTime = datetime.datetime.today().strftime("%Y%m%d_%H%M")
                    File_Name = "/Users/choiyoungmin/PycharmProjects/2020Y/Anal_Comp/Ananlyst_"+Comp_Name+"_"+DateTime+".txt"
                    if platform == 'darwin' :
                        output = open(File_Name, 'a+t')
                    else :
                        output = open(File_Name, 'a+t', encoding='utf-8')
                else :
                    DateTime_Win = datetime.datetime.today().strftime("%Y%m%d")
                    File_Name = "Analyze"+Comp_Name+"_"+DateTime_Win+".txt"
                    output = open(File_Name, 'a+t', encoding='utf-8')

                print("############################################################")
                output.write("############################################################\n")
                print(tabulate(Result_Comp_New[['회사명', 'ROE', 'PER', 'PBR', '최근PBR', '최근PER', 'PBR*PER=22.5', '최근EPS', '업종PER', '현재가', '예상시가', 'GAP', '외국인비율', 'Market', 'expcode', 'corpcode']].head(TOP_No),headers='keys',tablefmt='psql'))
                output.write("%s\n" % Result_Comp_New[['회사명', 'ROE', 'PER', '최근PBR', 'PBR', '최근PER', 'PBR*PER=22.5', '최근EPS', '업종PER', '현재가', '예상시가', 'GAP', '외국인비율', 'Market', 'expcode', 'corpcode']].head(TOP_No))
                Write_TEXT = Result_Comp_New[['회사명', 'ROE', 'PER', 'PBR', '최근PBR', '최근PER', 'PBR*PER=22.5', '최근EPS', '업종PER', '현재가', '예상시가', 'GAP', '외국인비율', 'Market', 'expcode', 'corpcode']].head(TOP_No)
                output.close()

                with open(File_Name, 'a+t') as f:
                    f.write(tabulate(Write_TEXT))

                output = open(File_Name, 'a+t')
                print("############################################################")

                output.write("\n")
                output.write("############################################################\n")
                print("# 기업명 : %s " % Result_Comp_New['회사명'].iloc[int(Choice_Comp)])
                output.write("# 기업명 : %s \n" % Result_Comp_New['회사명'].iloc[int(Choice_Comp)])
                print("############################################################")
                output.write("############################################################\n")
                output.write("\n")
                output.close() # ---> 아래부터는 각 function에서 열고 닫는다

                Dart_Exp_Code = Result_Comp_New['expcode'].iloc[int(Choice_Comp)]
                Dart_Corp_Code = Result_Comp_New['corpcode'].iloc[int(Choice_Comp)]
                Dart_Comp_Name = Result_Comp_New['회사명'].iloc[int(Choice_Comp)]
                Dart_Market_Name = Result_Comp_New['Market'].iloc[int(Choice_Comp)]

                #  교촌, 미스터블루 오류나고 있음....
                __Naver_CompInfo(Dart_Corp_Code, Dart_Exp_Code, File_Name)
                __Naver_BusinessType(Dart_Exp_Code, File_Name)
                __Naver_Anal(Dart_Exp_Code, File_Name)
                tail_1 = __Dart_Search_List(Dart_Corp_Code, Dart_Exp_Code, File_Name) # 공시보고 전체 리스트, return으로 보고서 number 전달
                input_rcept_no = tail_1.rcept_no.item()

                __Dart_Search_1st(Dart_Corp_Code, Dart_Exp_Code, File_Name)
                __Dart_Search_1st_Diff(Dart_Corp_Code, Dart_Exp_Code, File_Name)
                __Dart_Search_Stock(Dart_Corp_Code, Dart_Exp_Code, File_Name)
                __Dart_Search_Emp(Dart_Corp_Code, Dart_Exp_Code, File_Name)
                __Dart_Search_Report(Dart_Corp_Code, Dart_Exp_Code, File_Name)
                __Dart_Search_Report_Detail(Dart_Corp_Code, Dart_Exp_Code, File_Name) # 현금흐름표
                __Dart_Search_Attention(Dart_Corp_Code, Dart_Exp_Code, File_Name, input_rcept_no)
                # 필립 피셔의 15가지
                # 1. 향후 매출액이 상당히 늘어날만한 충분한 시장잠재력을 갖춘 제품/서비스가 있는가
                # 2. 경영진은 기존시장이 어려울 때 신규 모멘텀을 만들만한 신제품과 기술, 의지를 보유했는가
                # 3. 기업의 연구개발 노력은 회사규모를 감안했을 때 얼마나 생산적인가
                # 4. 평균수준 이상의 영업조직을 갖고있는가
                # 5. 영업이익률은 충분히 거두고 있는가
                # 6. 영업이익률 개선을 위해 무엇을 하고 있는가
                # 7. 돋보이는 노사관계를 갖고있는가
                # 8. 임원들간에 훌륭한 관계거 유지되고 있는가
                # 9. 능력있는, 두터운 경영진을 보유하고 있는가
                # 10 원가분석과 회계관리 능력은 얼마나 우수한가
                # 11 해당 업종에서 아주 특별한 의미를 지니는 별도의 사업부문을 갖고있으며,
                #    이는 경쟁업체에 비해 얼마나 뛰어난 기업인가를 알려주는 중요한 단서를 제공한는가?
                #    핵심전략을 기획하는 사업부와 같은 특수부서를 갖고있으며 뛰어난가
                # 12 이익을 바라보는 시각, 회사의 사업계획이 단기적인가, 장기적인가
                # 13 성장에 필요한 자금조달을 위해 가까운 미래에 증자할 계획이 있으며, 이로 인해 현재애 주주가 누리는 이익이 상당부분 희석될 가능성은 없는가
                # 14 경영진은 투자자들가 회사의 장점 뿐만이 아니라 문제에 관해서도 충분한 대화를 하는가
                # 15 경영진은 진실하고 정직한가

                # Make_Chart = 0
                Make_Chart = 1
                if Make_Chart ==1 : 
                    if Result_Comp_New['Market'].iloc[int(Choice_Comp)] == "코스피" :
                        __New_Make_Chart(Dart_Exp_Code, Dart_Comp_Name, "코스피", KOSPI_Date, KOSPI_List)
                    elif Result_Comp_New['Market'].iloc[int(Choice_Comp)] == "코스닥" :
                        __New_Make_Chart(Dart_Exp_Code, Dart_Comp_Name, "코스닥", KOSDAQ_Date, KOSDAQ_List)
                    else :
                        print("# Make Chart Error")
                print("# Naver Research Report")
                __Naver_Research(Dart_Comp_Name, Dart_Exp_Code, File_Name)

                # __NPS_Check()d
                # __Google_Upload(File_Name)

            else :
                print("# NOT Digit")
                break

    # File_Name = "/Users/choiyoungmin/PycharmProjects/2020Y/Anal_Comp/test.txt"

    print("# Program End")

############################################################
# Program Start
############################################################
############################################################
# 10 페이지 조회, 1페이지 당 50개, 총 500개
# ROE_Min = 30
# PER_Min, PER_Max = 1, 20
############################################################
# FOR Loop Start
# Market : 1
# Main_DF_Result : Sorting ROE...
# Main_DF_Result : Sorting ROE then Condition with ROE_Min...
# FOR Loop End
# FOR Loop Start
# Market : 2
# Main_DF_Result : Sorting ROE...
# Main_DF_Result : Sorting ROE then Condition with ROE_Min...
# FOR Loop End
############################################################
# Market Price List Getting...
# Market Page_Number :  1359
# Company : KOSPI
# Market Page_Number :  1039
# Company : KOSDAQ
# Market Price List Done...
############################################################
# TOP 30
# ROE_Min = 30
# PER_Min, PER_Max = 1, 20
#########################################################

4
<class 'str'>
# 다시 시작...
############################################################
+----+------------------+--------+-------+-------+-----------+-----------+----------------+-----------+-----------+----------+------------+--------+--------------+----------+-----------+------------+
|    | 회사명           |    ROE |   PER |   PBR |   최근PBR |   최근PER |   PBR*PER=22.5 | 최근EPS   |   업종PER | 현재가   | 예상시가   |    GAP |   외국인비율 | Market   |   expcode |   corpcode |
|----+------------------+--------+-------+-------+-----------+-----------+----------------+-----------+-----------+----------+------------+--------+--------------+----------+-----------+------------|
|  0 | 씨젠             | 125.91 | 11.25 |  8.66 |      8.72 |     11.25 |             98 | 9,617     |     18.98 | 108,200  | 182,530    |  168.7 |        17.52 | 코스닥   |    096530 |   00788773 |
|  1 | 멕아이씨에스     | 113.22 |  9.43 |  6.22 |      6.56 |      9.43 |             61 | 1,855     |     64.61 | 17,500   | 119,851    |  684.9

                report_nm        rcept_no  rcept_dt
0   [기재정정]사업보고서 (2020.12)  20210331000261  20210331
1         사업보고서 (2020.12)  20210319000531  20210319
2         분기보고서 (2020.09)  20201113000472  20201113
3         반기보고서 (2020.06)  20200814000070  20200814
4         분기보고서 (2020.03)  20200515002202  20200515
5         사업보고서 (2019.12)  20200330003946  20200330
6         분기보고서 (2019.09)  20191113000616  20191113
7         반기보고서 (2019.06)  20190813000595  20190813
8         분기보고서 (2019.03)  20190515000787  20190515
9         사업보고서 (2018.12)  20190401000954  20190401
10  [기재정정]반기보고서 (2018.06)  20190114000067  20190114
11  [기재정정]분기보고서 (2018.03)  20190114000061  20190114
12  [기재정정]사업보고서 (2017.12)  20190114000055  20190114
13        분기보고서 (2018.09)  20181114002720  20181114
14  [기재정정]사업보고서 (2017.12)  20180823000286  20180823
15  [기재정정]사업보고서 (2017.12)  20180814003042  20180814
16  [기재정정]분기보고서 (2018.03)  20180814003033  20180814
17        반기보고서 (2018.06)  20180814002991  20180814
18        분기

# 2015년 최대주주 현황
# 접수번호, 성명, 관계, 주식종류, 기초 소유 주식 지분율, 기말 소유 주식 지분율, 비고
          rcept_no   nm  relate  stock_knd bsis_posesn_stock_qota_rt trmend_posesn_stock_qota_rt     rm
0   20160330001076  박한오      본인  의결권 있는 주식                     24.20                       22.20      -
1   20160330001076  김재하   회사 임원          〃                      0.10                        0.10  장내 매수
2   20160330001076  이 효  관계회사임원          〃                      0.00                        0.00  장내 매도
3   20160330001076  김명희     배우자          〃                      1.00                        0.90      -
4   20160330001076  박준성      자녀          〃                      0.50                        0.40      -
5   20160330001076  박준영      자녀          〃                      0.20                        0.20      -
6   20160330001076  박유진      자녀          〃                      0.20                        0.20      -
7   20160330001076  박한이      동생          〃                      1.90                        1.70   

# 2015년 최대주주 변동 현황
# 접수번호, 변동일, 최대주주명, 지분율, 변동원인, 비고
         rcept_no change_on mxmm_shrholdr_nm qota_rt change_cause rm
0  20160330001076         -                -       -            -  -

# 2016년 최대주주 변동 현황
# 접수번호, 변동일, 최대주주명, 지분율, 변동원인, 비고
         rcept_no change_on mxmm_shrholdr_nm qota_rt change_cause rm
0  20170331001784         -                -       -            -  -

# 2017년 최대주주 변동 현황
# 접수번호, 변동일, 최대주주명, 지분율, 변동원인, 비고
         rcept_no change_on mxmm_shrholdr_nm qota_rt change_cause rm
0  20190114000055         -                -       -            -  -

# 2018년 최대주주 변동 현황
# 접수번호, 변동일, 최대주주명, 지분율, 변동원인, 비고
         rcept_no change_on mxmm_shrholdr_nm qota_rt change_cause rm
0  20190401000954         -                -       -            -  -

# 2019년 최대주주 변동 현황
# 접수번호, 변동일, 최대주주명, 지분율, 변동원인, 비고
         rcept_no change_on mxmm_shrholdr_nm qota_rt change_cause rm
0  20200330003946         -                -       -            -  -

# 2020년 최대주주 변동 현황
# 접수번호, 변동일, 최대주주명, 지

...

############################################################
# 재무재표 정제해서 출력.......
############################################################
   account_nm                thstrm_dt   thstrm_amount                frmtrm_dt   frmtrm_amount             bfefrmtrm_dt bfefrmtrm_amount
10       영업이익  2017.01.01 ~ 2017.12.31  21,702,287,357  2016.01.01 ~ 2016.12.31  34,249,464,048  2015.01.01 ~ 2015.12.31   33,317,086,542
23       영업이익  2017.01.01 ~ 2017.12.31  46,171,105,785  2016.01.01 ~ 2016.12.31  56,117,585,290  2015.01.01 ~ 2015.12.31   43,659,739,007
    account_nm                thstrm_dt   thstrm_amount                frmtrm_dt   frmtrm_amount             bfefrmtrm_dt bfefrmtrm_amount
11  법인세차감전 순이익  2017.01.01 ~ 2017.12.31  19,430,303,923  2016.01.01 ~ 2016.12.31  32,482,990,896  2015.01.01 ~ 2015.12.31   26,781,591,466
24  법인세차감전 순이익  2017.01.01 ~ 2017.12.31  31,900,767,788  2016.01.01 ~ 2016.12.31  42,536,754,576  2015.01.01 ~ 2015.12.31   50,860,941,744
   account_nm            

############################################################
# 재무재표 정제해서 출력.......
############################################################
   account_nm                thstrm_dt   thstrm_amount                frmtrm_dt   frmtrm_amount             bfefrmtrm_dt bfefrmtrm_amount
10       영업이익  2018.01.01 ~ 2018.12.31  30,975,989,508  2017.01.01 ~ 2017.12.31  21,702,287,357  2016.01.01 ~ 2016.12.31   34,249,464,048
23       영업이익  2018.01.01 ~ 2018.12.31  54,865,258,710  2017.01.01 ~ 2017.12.31  46,171,105,785  2016.01.01 ~ 2016.12.31   56,117,585,290
    account_nm                thstrm_dt   thstrm_amount                frmtrm_dt   frmtrm_amount             bfefrmtrm_dt bfefrmtrm_amount
11  법인세차감전 순이익  2018.01.01 ~ 2018.12.31  18,023,590,383  2017.01.01 ~ 2017.12.31  19,430,303,923  2016.01.01 ~ 2016.12.31   32,482,990,896
24  법인세차감전 순이익  2018.01.01 ~ 2018.12.31  28,791,491,801  2017.01.01 ~ 2017.12.31  31,900,767,788  2016.01.01 ~ 2016.12.31   42,536,754,576
   account_nm            

############################################################
# 재무재표 정제해서 출력.......
############################################################
   account_nm                thstrm_dt   thstrm_amount                frmtrm_dt   frmtrm_amount             bfefrmtrm_dt bfefrmtrm_amount
10       영업이익  2019.01.01 ~ 2019.12.31  42,901,163,935  2018.01.01 ~ 2018.12.31  30,975,989,508  2017.01.01 ~ 2017.12.31                -
23       영업이익  2019.01.01 ~ 2019.12.31  58,141,274,260  2018.01.01 ~ 2018.12.31  54,865,258,710  2017.01.01 ~ 2017.12.31                -
    account_nm                thstrm_dt   thstrm_amount                frmtrm_dt   frmtrm_amount             bfefrmtrm_dt bfefrmtrm_amount
11  법인세차감전 순이익  2019.01.01 ~ 2019.12.31  29,329,914,189  2018.01.01 ~ 2018.12.31  18,023,590,383  2017.01.01 ~ 2017.12.31                -
24  법인세차감전 순이익  2019.01.01 ~ 2019.12.31  32,995,597,532  2018.01.01 ~ 2018.12.31  28,791,491,801  2017.01.01 ~ 2017.12.31                -
   account_nm            

############################################################
# 재무재표 정제해서 출력.......
############################################################
   account_nm                thstrm_dt   thstrm_amount                frmtrm_dt   frmtrm_amount             bfefrmtrm_dt bfefrmtrm_amount
10       영업이익  2020.01.01 ~ 2020.12.31  98,097,937,707  2019.01.01 ~ 2019.12.31  42,901,163,935  2018.01.01 ~ 2018.12.31                -
23       영업이익  2020.01.01 ~ 2020.12.31  93,101,265,802  2019.01.01 ~ 2019.12.31  58,141,274,260  2018.01.01 ~ 2018.12.31                -
    account_nm                thstrm_dt    thstrm_amount                frmtrm_dt   frmtrm_amount             bfefrmtrm_dt bfefrmtrm_amount
11  법인세차감전 순이익  2020.01.01 ~ 2020.12.31  103,059,273,648  2019.01.01 ~ 2019.12.31  29,329,914,189  2018.01.01 ~ 2018.12.31                -
24  법인세차감전 순이익  2020.01.01 ~ 2020.12.31  100,434,839,137  2019.01.01 ~ 2019.12.31  32,995,597,532  2018.01.01 ~ 2018.12.31                -
   account_nm         

IndexError: index 0 is out of bounds for axis 0 with size 0